In [31]:
run_dash_mode = True
run_dash_mode = False
use_data_subsets = True
use_data_subsets = False
dash_app_name = "JupyterDash"
dash_app_name = "Dash"
dev_mode = True
# dev_mode = False
# prod_mode = True
prod_mode = False
if dash_app_name == "JupyterDash":
    dev_mode = False
if prod_mode:
    dev_mode = False
    # deactivate; source venv_maine_analysis/bin/activate; jupyter nbconvert maine_analysis.ipynb --to python; deactivate; source venv_maine_analysis_prod/bin/activate; python maine_analysis.py
    # DEST=deploy_mobile; cp maine_analysis.py ${DEST}/main.py; cp -r exports ${DEST}; cp deploy.sh ${DEST}; cp Dockerfile ${DEST}; cp requirements_prod_long.txt ${DEST}
# test: jupyter nbconvert maine_analysis.ipynb --to python; python maine_analysis.py
# 


In [32]:
if True: # define constants & paths
    legislators_glob = (
        "data/openstates/people/data/me/*/*yml")
    legislature_metadata = "data/openstates/people/data/me/legislature.yml"
    bills_glob = "data/ME/*/*json"
    session_data_file = "data/ME/session_data.csv"
    if prod_mode:
        exports = "."
    else:
        exports = "exports"

In [33]:
if True: # import packages; setup logging
    if not run_dash_mode: # dev imports
        import yaml
        import glob
        import uuid
        import re
        import pprint
        import csv
        import datetime
        import math
        import textwrap
        import heapq
        from unidecode import unidecode
        import numpy as np
        from collections import defaultdict

        import kaleido

    if True: # import always

        import json
        import copy
        import sys
        import os

        if dev_mode:
            from icecream import ic
        else:
            def ic(*args, **kwargs):
                return args

        if not prod_mode:
            from jupyter_dash import JupyterDash
           
        import pandas as pd

        import plotly.express as px
        
 
        from dash import dcc, html, Dash, dash_table

        import dash_bootstrap_components as dbc
        from dash.dependencies import Input, Output

        import logging # Basic config below is used for MS Code compatibility
        LOG = logging.getLogger(__name__)
        LOG.setLevel(logging.DEBUG)
        # if not run_dash_mode:
        logging.basicConfig(stream=sys.stderr)
        # else:
        #     logging.basicConfig(filename='dash.log')

        LOG.info("Logging (info) has been initialized")
        LOG.debug("Logging (debug) has been initialized")
    dash_app = globals()[dash_app_name]

INFO:__main__:Logging (info) has been initialized
DEBUG:__main__:Logging (debug) has been initialized


In [34]:
if True: # define & test functions
    # @TODO: import/export df not super clean
    if True: 
        def export_df(dataframe=None, df=None, filename=None, index=True):
            if dataframe is not None:
                df = globals()[dataframe]
                filename = dataframe
            filename = os.path.join(exports, f"{filename}.csv")
            df.to_csv(filename, index=index)

        def import_df(dataframe):
            filename = os.path.join(exports, f"{dataframe}.csv")
            df = pd.read_csv(filename, index_col=0)
            return df

        def export_dict(dictionary_name):
            dictionary = globals()[dictionary_name]
            filename = os.path.join(exports, f"{dictionary_name}.json")
            with open(filename, 'w') as _file:
                json.dump(dictionary, _file)

        def import_dict(dictionary_name):
            filename = os.path.join(exports, f"{dictionary_name}.json")
            with open(filename) as _file:
                result = json.load(_file)
            return result

        def get_party_lean(party, polarization):
            if party == "Democratic":
                party_lean = "Democratic"
            elif party == "Republican":
                party_lean = "Republican"
            elif polarization < 0.5:
                party_lean = "Democratic"
            else:
                party_lean = "Republican"
            return party_lean

        class SimpleDashTable():

            def update_table_data(self):
                # begin update: 
                if self.column_order: 
                    self.delete_columns = [
                        column for column in self.df.columns 
                        if column not in set(self.column_order)]
                if self.delete_columns:
                    self.df = self.df.drop(columns=self.delete_columns)
                if self.column_order: 
                    self.df = self.df[self.column_order]
                if self.column_specification == None:
                    self.column_specification = self.default_column_specification
                self.columns = [
                    self.column_specification(
                        id, name, 
                        self.column_name_transform(name),
                        column_specs=self.column_specs) 
                    for id, name in enumerate(self.df.columns)]
                original_columns = self.df.columns
                self.df.columns = (
                    [f"c{id}" for id, name in enumerate(self.df.columns)])
                self.data = self.df.to_dict('records')
                self.df.columns = original_columns
                self.column_lookup = (
                    {name: f"c{id}" for id, name in enumerate(self.df.columns)})

            def update_tables(self):
                table_dict = {
                    "data": self.data, 
                    "columns": self.columns,
                }
                if self.table_style is not None:
                    table_dict["style_cell"] = self.table_style
                self.static_table = dash_table.DataTable(**table_dict)

                if self.dash_id == None:
                    self.sortable_table = dcc.Markdown("A sortable table requires an dash_id.")
                else:
                    # dash_table.DataTable(
                    sortable_table_dict = copy.copy(table_dict)
                    sortable_table_default_options = dict(
                        id=self.dash_id,
                        filter_action="native",
                        filter_options={"case":"insensitive"},
                        sort_action="native",
                        sort_mode="single",
                        selected_columns=[],
                        selected_rows=[],
                        page_action="native",
                        page_current= 0,
                        page_size= 10,
                        )
                    if self.empty:
                        sortable_table_dict["data"] = []
                        sortable_table_dict["columns"] = []
                    sortable_table_dict.update(sortable_table_default_options)
                    sortable_table_dict.update(self.sortable_table_options)
                    self.sortable_table = dash_table.DataTable(**sortable_table_dict)

            def __init__(
                self, df, 
                table_style=None, 
                column_specification=None,
                delete_columns=[], 
                column_order=[],
                column_specs={},
                column_name_transform=None,
                dash_id=None,
                sortable_table_options={},
                empty=False,
                ):
                self.df = df.copy()
                self.table_style = table_style
                self.column_specification = column_specification
                self.delete_columns = delete_columns
                self.column_order = column_order
                self.column_specs = column_specs
                self.column_name_transform = column_name_transform
                self.dash_id = dash_id  
                self.id = dash_id
                self.sortable_table_options=sortable_table_options
                self.empty = empty

                self.update_table_data()
                self.update_tables()
                
            @staticmethod
            def default_column_specification(id, full_name, short_name, column_specs={}):
                result = {"name": short_name, "id": f"c{id}"}
                if full_name in column_specs.keys():
                    result.update(column_specs[full_name])
                return result
                    # "type": "numeric",
                    # "format": dash_table.FormatTemplate.percentage(1)}
            pass

        def get_voter_polarization_fig(df):
            fig = px.scatter(
                    df,
                    x="Polarization", 
                    y="Party agreement",
                    color="Party",
                    hover_data = {
                        'Name': True,
                        'Party': True,
                        'Seat': True,
                        'Polarization': ":,.0%",
                        'Party agreement': ":,.0%",
                    },
                )
            fig.update_layout(yaxis_tickformat = ',.0%')
            fig.update_layout(xaxis_tickformat = ',.0%')
            return fig

        def get_vote_polarization_fig(df):
            fig = px.scatter(
                df,
                x="Republican yes", 
                y = "Democratic yes",
                symbol="Session",
                color="Party polarization",
                color_continuous_scale=["blue", "red"],
                hover_data = {
                    'title_w_breaks': True,
                    'Date': True,
                    'Session': True,
                    'Party polarization': ":,.0%",
                },   
                labels={"title_w_breaks": "Title"},
            )
            fig.update_layout(yaxis_tickformat=',.0%')
            fig.update_layout(xaxis_tickformat=',.0%')
            fig.update_coloraxes(colorbar_tickformat=',.0%')
            fig.update_layout(legend_orientation="h")
            return fig

    if not run_dash_mode:
        def splitall(path):
            """
            Split path into a list of component directories.
            https://www.oreilly.com/library/view/python-cookbook/0596001673/ch04s16.html
            """
            allparts = []
            while 1:
                parts = os.path.split(path)
                if parts[0] == path:  # sentinel for absolute paths
                    allparts.insert(0, parts[0])
                    break
                elif parts[1] == path:  # sentinel for relative paths
                    allparts.insert(0, parts[1])
                    break
                else:
                    path = parts[0]
                    allparts.insert(0, parts[1])
            return allparts

        def extract_word(text):
            # print( "Input Text::{}".format(text))
            regex = r"(\w|\s)*"
            matches = re.finditer(regex, text, re.DOTALL)
            newstr = ''
            for matchNum, match in enumerate(matches):
                matchNum = matchNum + 1
                newstr = newstr + match.group()
            # print ("Output Text::{}".format(newstr))
            return newstr

        def munge_string(string, upper=True):
            # string_types.add(type(string))
            # return string
            string = unidecode(extract_word(string))
            if upper:
                string = string.upper()
            string.replace("'","")
            string.replace("-","")
            return string

        def party_line_vote_test(vote_result, threshold=0.9):
            # Straight Party-Line: All Republicans took one position, all Democrats took the other.
            # Near Party-Line: At least {threshold} of one or both parties voted the same way.
            # Not Party-Line: Neither party had at least {threshold} of members vote the same way.
            # Bipartisan: at least {threshold} of both parties voted the same way
            threshold_ratio = (threshold)/(1-threshold)
            vote_left = vote_result["Democratic"]
            vote_right = vote_result["Republican"]
            bipart_yes = (
                (vote_left["yes"] > threshold_ratio * vote_left["no"])
                and (vote_right["yes"] > threshold_ratio * vote_right["no"]))
            bipart_no = (
                (vote_left["no"] > threshold_ratio * vote_left["yes"])
                and (vote_right["no"] > threshold_ratio * vote_right["yes"]))
            if bipart_yes or bipart_no:
                return "bipartisan"
            party_line_left = ((vote_left["yes"] == 0) != (vote_left["no"] == 0))
            party_line_right = ((vote_right["yes"] == 0) != (vote_right["no"] == 0))
            if party_line_left and party_line_right:
                return "straight_party_line"
            near_party_line_left = (
                (vote_left["yes"] > threshold_ratio * vote_left["no"])
                or (vote_left["no"] > threshold_ratio * vote_left["yes"]))
            near_party_line_right = (
                (vote_right["yes"] > threshold_ratio * vote_right["no"])
                or (vote_right["no"] > threshold_ratio * vote_right["yes"]))
            if (party_line_left or party_line_right
                or near_party_line_left or near_party_line_right):
                return "near_party_line"
            return "not_party_line"

        def test_party_line_vote_test():
            test = {
                'Democratic': {
                    'yes': 16, 
                    'no': 1}, 
                'Republican': {
                    'yes': 18, 
                    'no': 1}
            }
            assert party_line_vote_test(test) == "bipartisan"

            test = {
                'Democratic': {
                    'yes': 16, 
                    'no': 0}, 
                'Republican': {
                    'yes': 11, 
                    'no': 1}
            }
            assert party_line_vote_test(test) == "bipartisan"

            test = {
                'Democratic': {
                    'yes': 16, 
                    'no': 0}, 
                'Republican': {
                    'yes': 8, 
                    'no': 0}
            }
            assert party_line_vote_test(test) == "bipartisan"

            test = {
                'Democratic': {
                    'yes': 16, 
                    'no': 0}, 
                'Republican': {
                    'yes': 0, 
                    'no': 8}
            }
            assert party_line_vote_test(test) == "straight_party_line"

            test = {
                'Democratic': {
                    'yes': 0, 
                    'no': 20}, 
                'Republican': {
                    'yes': 5, 
                    'no': 0}
            }
            assert party_line_vote_test(test) == "straight_party_line"

            test = {
                'Democratic': {
                    'yes': 16, 
                    'no': 1}, 
                'Republican': {
                    'yes': 0, 
                    'no': 8}
            }
            assert party_line_vote_test(test) == "near_party_line"

            test = {
                'Democratic': {
                    'yes': 16, 
                    'no': 0}, 
                'Republican': {
                    'yes': 1, 
                    'no': 11}
            }
            assert party_line_vote_test(test) == "near_party_line"

            test = {
                'Democratic': {
                    'yes': 16, 
                    'no': 0}, 
                'Republican': {
                    'yes': 4, 
                    'no': 8}
            }
            assert party_line_vote_test(test) == "near_party_line"

            test = {
                'Democratic': {
                    'yes': 8, 
                    'no': 1}, 
                'Republican': {
                    'yes': 1, 
                    'no': 8}
            }
            assert party_line_vote_test(test) == "not_party_line"

            test = {
                'Democratic': {
                    'yes': 8, 
                    'no': 1}, 
                'Republican': {
                    'yes': 8, 
                    'no': 1}
            }
            assert party_line_vote_test(test) == "not_party_line"

        test_party_line_vote_test()

        def party_line_voter_test(legislator_vote, vote_summary, threshold=0.9):
            # party_line_left: agreed with left majority above threshold.
            # party_line_right: agreed with right majority above threshold.
            # bipartisan: agreed with both parties above threshold.
            # not_party_line: either no party line, or voted against both parties
            if (legislator_vote["option"] != "yes"
                and legislator_vote["option"] != "no"):
                # print("other")
                return "other"
            threshold_ratio = (threshold)/(1-threshold)
            vote_left = vote_summary["Democratic"]
            vote_right = vote_summary["Republican"]
            party_line_left, party_line_right = False, False
            if legislator_vote["option"] == "yes":
                frac_left = vote_left["yes"] / (vote_left["yes"] + vote_left["no"])
                frac_right = vote_right["yes"] / (vote_right["yes"] + vote_right["no"])
            if legislator_vote["option"] == "no":
                frac_left = vote_left["no"] / (vote_left["yes"] + vote_left["no"])
                frac_right = vote_right["no"] / (vote_right["yes"] + vote_right["no"])
            frac_other = ((1-frac_left) + (1-frac_right)) / 2
            # print([frac_left, frac_right, frac_other])
            if (vote_left["yes"] > threshold_ratio * vote_left["no"]
                and legislator_vote["option"] == "yes"):
                party_line_left = True
            if (vote_left["no"] > threshold_ratio * vote_left["yes"]
                and legislator_vote["option"] == "no"):
                party_line_left = True
            if (vote_right["yes"] > threshold_ratio * vote_right["no"]
                and legislator_vote["option"] == "yes"):
                party_line_right = True
            if (vote_right["no"] > threshold_ratio * vote_right["yes"]
                and legislator_vote["option"] == "no"):
                party_line_right = True
            if party_line_left and party_line_right:
                # print("bipartisan")
                return "bipartisan"
            if party_line_left:
                # print("party_line_left")
                return "party_line_left"
            if party_line_right:
                # print("party_line_right")
                return "party_line_right"
            # print("not_party_line")
            return "not_party_line"

        def test_party_line_voter_test():

            legislator_vote = {'option': 'other', 'voter_name': 'ACKLEY of Monmouth'}
            vote_summary = {
                'Democratic': {
                    'yes': 16, 
                    'no': 1}, 
                'Republican': {
                    'yes': 18, 
                    'no': 1}
            }
            assert party_line_voter_test(legislator_vote, vote_summary) == "other"

            legislator_vote = {'option': 'yes', 'voter_name': 'ACKLEY of Monmouth'}
            vote_summary = {
                'Democratic': {
                    'yes': 16, 
                    'no': 1}, 
                'Republican': {
                    'yes': 18, 
                    'no': 1}
            }
            # 
            assert party_line_voter_test(legislator_vote, vote_summary) == "bipartisan"

            legislator_vote = {'option': 'no', 'voter_name': 'ACKLEY of Monmouth'}
            vote_summary = {
                'Democratic': {
                    'yes': 0, 
                    'no': 16}, 
                'Republican': {
                    'yes': 1, 
                    'no': 17}
            }
            assert party_line_voter_test(legislator_vote, vote_summary) == "bipartisan"

            legislator_vote = {'option': 'yes', 'voter_name': 'ACKLEY of Monmouth'}
            vote_summary = {
                'Democratic': {
                    'yes': 0, 
                    'no': 16}, 
                'Republican': {
                    'yes': 1, 
                    'no': 17}
            }
            assert party_line_voter_test(legislator_vote, vote_summary) == "not_party_line"

            legislator_vote = {'option': 'yes', 'voter_name': 'ACKLEY of Monmouth'}
            vote_summary = {
                'Democratic': {
                    'yes': 16, 
                    'no': 0}, 
                'Republican': {
                    'yes': 0, 
                    'no': 8}
            }
            assert party_line_voter_test(legislator_vote, vote_summary) == "party_line_left"

            legislator_vote = {'option': 'no', 'voter_name': 'ACKLEY of Monmouth'}
            vote_summary = {
                'Democratic': {
                    'yes': 16, 
                    'no': 0}, 
                'Republican': {
                    'yes': 0, 
                    'no': 8}
            }
            assert party_line_voter_test(legislator_vote, vote_summary) == "party_line_right"

            legislator_vote = {'option': 'no', 'voter_name': 'ACKLEY of Monmouth'}
            vote_summary = {
                'Democratic': {
                    'yes': 10, 
                    'no': 20}, 
                'Republican': {
                    'yes': 5, 
                    'no': 0}
            }
            assert party_line_voter_test(legislator_vote, vote_summary) == "not_party_line"

            legislator_vote = {'option': 'no', 'voter_name': 'ACKLEY of Monmouth'}
            vote_summary = {
                'Democratic': {
                    'yes': 8, 
                    'no': 1}, 
                'Republican': {
                    'yes': 1, 
                    'no': 8}
            }
            assert party_line_voter_test(legislator_vote, vote_summary) == "not_party_line"

            legislator_vote = {'option': 'yes', 'voter_name': 'ACKLEY of Monmouth'}
            vote_summary = {
                'Democratic': {
                    'yes': 8, 
                    'no': 1}, 
                'Republican': {
                    'yes': 1, 
                    'no': 8}
            }
            assert party_line_voter_test(legislator_vote, vote_summary) == "not_party_line"
        test_party_line_voter_test()

        def square_to_diamond(result):
            i, j = result
            if i >= j: # right triangle
                a = 1.
                b = -1.
                c = 2
                d = 0
                e = 0.
                f = -1.
            # else: # left triangle
            #     a = 1.
            #     b = -1.
            #     c = 0.5
            #     d = 1.5
            #     e = 0.
            #     f = -1.
            return [e + a*i + b*j, f + c*i + d*j]

        def polarization_metric_1(
            votes_dict, party0, party1, 
            circular=True, 
            diamond=False):
            yes0 = votes_dict[party0].get("yes", 0)
            no0 = votes_dict[party0].get("no", 0)
            if yes0 == 0 and no0 == 0:
                raise KeyError("Error: no yes or no votes for {party0}: {votes_dict}")
            ratio0 = yes0 / (yes0 + no0)
            yes1 = votes_dict[party1].get("yes", 0)
            no1 = votes_dict[party1].get("no", 0)
            if ((yes1 == 0 and no1 == 0)
                or party1 not in votes_dict):
                    return max(ratio0, 1-ratio0)
            ratio1 = yes1 / (yes1 + no1)
            if ratio0 < 0.5:
                ratio0, ratio1 = 1 - ratio0, (1-ratio1)
            if ratio0 > ratio1:
                ratio_of_ratios = ratio1 / ratio0
            else: 
                ratio_of_ratios = ratio0 / ratio1
            result = (ratio0, 
                ratio_of_ratios * ratio0
                + (1 - ratio_of_ratios) * (1 - ratio0)
                )
            if circular:
                return square_to_pie(result)
            elif diamond:
                return square_to_diamond(result)
            else:
                return result

        ROUNDING_ERROR = 1E-6
        def test_polarization_metric_1_helper(vote_summary, party0, party1, target):
            # def test_metric_helper(metric_function, *args, **kwargs):
            # print(vote_summary)
            # print(party0)
            # print(party1)
            # print(f"expected: {target}")
            result = polarization_metric_1(vote_summary, party0, party1, circular=False)
            # print(f"actual: {result}")
            # print()
            assert abs(target[0] -  result[0]) < ROUNDING_ERROR
            assert abs(target[1] -  result[1]) < ROUNDING_ERROR

        # def test_polarization_metric_1_helper(*args, **kwargs):
        #     test_metric_helper(polarization_metric_1, *args, **kwargs)

        def test_polarization_metric_1():
            # good
            vote_summary = {
                'Democratic': {
                    'yes': 16, 
                    'no': 0}, 
                'Republican': {
                    'yes': 0, 
                    'no': 18}
            }
            test_polarization_metric_1_helper(vote_summary, "Democratic", "Republican", [1.0, 0.0])

            # good
            vote_summary = {
                'Democratic': {
                    'yes': 16, 
                    'no': 0}, 
                'Republican': {
                    'yes': 0, 
                    'no': 18}
            }
            test_polarization_metric_1_helper(vote_summary, "Republican", "Democratic", [1.0, 0.0])

            # good
            vote_summary = {
                'Democratic': {
                    'yes': 16, 
                    'no': 0}, 
                'Republican': {
                    'yes': 18, 
                    'no': 0}
            }
            test_polarization_metric_1_helper(vote_summary, "Republican", "Democratic", [1.0, 1.0])

            # good
            vote_summary = {
                'Democratic': {
                    'yes': 8, 
                    'no': 8}, 
                'Republican': {
                    'yes': 9, 
                    'no': 9}
            }
            test_polarization_metric_1_helper(vote_summary, "Republican", "Democratic", [0.5, 0.5])

            # good
            vote_summary = {
                'Democratic': {
                    'yes': 3, 
                    'no': 1}, 
                'Republican': {
                    'yes': 1, 
                    'no': 3}
            }
            test_polarization_metric_1_helper(
                vote_summary, "Republican", "Democratic", 
                [0.75, 1.0/3 * 0.75 + 2.0/3 * 0.25])

            # good
            vote_summary = {
                'Democratic': {
                    'yes': 3, 
                    'no': 0}, 
                'Republican': {
                    'yes': 1, 
                    'no': 3}
            }
            test_polarization_metric_1_helper(vote_summary, "Republican", "Democratic", [0.75, .25])

            vote_summary = {
                'Democratic': {
                    'yes': 3, 
                    'no': 0}, 
                'Republican': {
                    'yes': 1, 
                    'no': 3}
            }
            test_polarization_metric_1_helper(vote_summary, "Democratic", "Republican", [1.0, 0.25])

        test_polarization_metric_1()

        def _stretch_unanimity(result):
            x, y = result
            y = (y - 0.5) * 2
            return [x, y]

        def polarization_metric_2(
            votes_dict, party0, party1, 
            transform=None):
            """Returns [polarization, unanimity]"""
            if transform == None:
                transform = lambda x: x
            yes0 = votes_dict[party0].get("yes", 0)
            no0 = votes_dict[party0].get("no", 0)
            if yes0 == 0 and no0 == 0:
                raise KeyError("Error: no yes or no votes for {party0}: {votes_dict}")
            yes_fraction_0 = yes0 / (yes0 + no0)
            unanimity0 = max(yes_fraction_0, 1-yes_fraction_0)
            if party1 not in votes_dict:
                return transform([1, unanimity0, yes_fraction_0, 0])
            yes1 = votes_dict[party1].get("yes", 0)
            no1 = votes_dict[party1].get("no", 0)
            if yes1 == 0 and no1 == 0:
                return transform([1, unanimity0, yes_fraction_0, 0])
            yes_fraction_1 = yes1 / (yes1 + no1)
            polarization = abs(yes_fraction_0 - yes_fraction_1)
            party_polarization = yes_fraction_0 - yes_fraction_1
            return transform([polarization, unanimity0, yes_fraction_0, yes_fraction_1, party_polarization])

        ROUNDING_ERROR = 1E-6
        def test_polarization_metric_2_helper(vote_summary, party0, party1, target):
            # print(vote_summary)
            # print(party0)
            # print(party1)
            # print(f"expected: {target}")
            result = polarization_metric_2(vote_summary, party0, party1, transform=None)
            # print(f"actual: {result}")
            # print()
            assert abs(target[0] -  result[0]) < ROUNDING_ERROR
            assert abs(target[1] -  result[1]) < ROUNDING_ERROR

        def test_polarization_metric_2():
            # good
            vote_summary = {
                'Democratic': {
                    'yes': 16, 
                    'no': 0}, 
                'Republican': {
                    'yes': 0, 
                    'no': 18}
            }
            test_polarization_metric_2_helper(vote_summary, "Democratic", "Republican", [1.0, 1.0])
            test_polarization_metric_2_helper(vote_summary, "Republican", "Democratic", [1.0, 1.0])

            # good
            vote_summary = {
                'Democratic': {
                    'yes': 16, 
                    'no': 0}, 
                'Republican': {
                    'yes': 18, 
                    'no': 0}
            }
            test_polarization_metric_2_helper(vote_summary, "Republican", "Democratic", [0.0, 1.0])

            # good
            vote_summary = {
                'Democratic': {
                    'yes': 8, 
                    'no': 8}, 
                'Republican': {
                    'yes': 9, 
                    'no': 9}
            }
            test_polarization_metric_2_helper(vote_summary, "Republican", "Democratic", [0.0, 0.5])

            # good
            vote_summary = {
                'Democratic': {
                    'yes': 3, 
                    'no': 1}, 
                'Republican': {
                    'yes': 1, 
                    'no': 3}
            }
            test_polarization_metric_2_helper(vote_summary, "Republican", "Democratic", [0.5, 0.75])

            # good
            vote_summary = {
                'Democratic': {
                    'yes': 3, 
                    'no': 0}, 
                'Republican': {
                    'yes': 1, 
                    'no': 3}
            }
            test_polarization_metric_2_helper(vote_summary, "Republican", "Democratic", [0.75, .75])
            test_polarization_metric_2_helper(vote_summary, "Democratic", "Republican", [.75, 1])

        test_polarization_metric_2()

        def yes_polarization(vote_summary, transform=None):
            """Returns [polarization, party_agreement]"""
            if transform == None:
                transform = lambda x: x
            party0, party1 = "Democratic", "Republican"
            yes0 = vote_summary[party0].get("yes", 0)
            no0 = vote_summary[party0].get("no", 0)
            yes1 = vote_summary[party1].get("yes", 0)
            no1 = vote_summary[party1].get("no", 0)
            if (not yes0 and not no0):
                unanimity0 = 0
            else:
                unanimity0 = yes0 / (yes0 + no0) 
            if (not yes1 and not no1):
                unanimity1 = 0
            else:
                unanimity1 = yes1 / (yes1 + no1) 
            return unanimity1 - unanimity0

        ROUNDING_ERROR = 1E-6
        def test_yes_polarization_helper(vote_summary, target):
            # print(vote_summary.keys())
            # print(vote_summary)
            # print(f"expected: {target}")
            result = yes_polarization(vote_summary)
            # print(f"actual: {result}")
            # print()
            if type(target) == list:
                for i in range(len(target)):
                    assert abs(target[i] -  result[i]) < ROUNDING_ERROR
            else:
                assert abs(target - result) < ROUNDING_ERROR

        def test_yes_polarization():

            vote_summary = {
                'Democratic': {
                    'yes': 16, 
                    'no': 0}, 
                'Republican': {
                    'yes': 18, 
                    'no': 0}
            }
            # 
            test_yes_polarization_helper(vote_summary, 0.0)

            vote_summary = {
                'Republican': {
                    'yes': 3, 
                    'no': 1}, 
                'Democratic': {
                    'yes': 0, 
                    'no': 18}
            }
            test_yes_polarization_helper(vote_summary, 0.75)

            vote_summary = {
                'Democratic': {
                    'yes': 3, 
                    'no': 1}, 
                'Republican': {
                    'yes': 0, 
                    'no': 18}
            }
            test_yes_polarization_helper(vote_summary, -0.75)

            vote_summary = {
                'Democratic': {
                    'yes': 3, 
                    'no': 3}, 
                'Republican': {
                    'yes': 18, 
                    'no': 18}
            }
            # 
            test_yes_polarization_helper(vote_summary, 0.0)

        test_yes_polarization()

        def voter_polarization(legislator_vote, vote_summary, transform=None):
            """Returns [polarization, party_agreement]"""
            if (legislator_vote["option"] != "yes"
                and legislator_vote["option"] != "no"):
                # print("other")
                raise ValueError("Vote must be 'yes' or 'no' to determine polarization")
            # return [0,0]
            if transform == None:
                transform = lambda x: x
            party0, party1 = "Democratic", "Republican"
            yes0 = vote_summary[party0].get("yes", 0)
            no0 = vote_summary[party0].get("no", 0)
            yes1 = vote_summary[party1].get("yes", 0)
            no1 = vote_summary[party1].get("no", 0)
            if (not yes0 and not no0):
                unanimity0 = 0
            else:
                unanimity0 = yes0 / (yes0 + no0) 
            if (not yes1 and not no1):
                unanimity1 = 0
            else:
                unanimity1 = yes1 / (yes1 + no1) 
            if legislator_vote["option"] == "yes":
                polarization = unanimity1 - unanimity0
                if unanimity0 > 0.5:
                    democratic_votes = 1
                    democratic_majority = 1
                elif unanimity0 == 0.5:
                    democratic_votes = 0
                    democratic_majority = 0
                else:
                    democratic_votes = 1
                    democratic_majority = 0
                if unanimity1 > 0.5:
                    republican_votes = 1
                    republican_majority = 1
                elif unanimity1 == 0.5:
                    republican_votes = 0
                    republican_majority = 0
                else:
                    republican_votes = 1
                    republican_majority = 0
                if polarization < 0:
                    party_agreement = unanimity0
                else:
                    party_agreement = unanimity1
            else:
                if unanimity0 > 0.5:
                    democratic_votes = 1
                    democratic_majority = 0
                elif unanimity0 == 0.5:
                    democratic_votes = 0
                    democratic_majority = 0
                else:
                    democratic_votes = 1
                    democratic_majority = 1
                if unanimity1 > 0.5:
                    republican_votes = 1
                    republican_majority = 0
                elif unanimity1 == 0.5:
                    republican_votes = 0
                    republican_majority = 0
                else:
                    republican_votes = 1
                    republican_majority = 1
                polarization = (1 - unanimity1) - (1 - unanimity0)
                if polarization < 0:
                    party_agreement = 1 - unanimity0
                else:
                    party_agreement = 1 - unanimity1
            return transform([
                polarization, party_agreement, 
                democratic_votes, democratic_majority, 
                republican_votes, republican_majority, 
                unanimity0, unanimity1,
                ])

        ROUNDING_ERROR = 1E-6
        def test_voter_polarization_helper(legislator_vote, vote_summary, target):
            # print(legislator_vote)
            # print(vote_summary.keys())
            # print(vote_summary)
            # print(f"expected: {target}")
            result = voter_polarization(legislator_vote, vote_summary)
            # print(f"actual: {result}")
            # print()
            for i in range(len(target)):
                assert abs(target[i] -  result[i]) < ROUNDING_ERROR

        def test_voter_polarization():

            vote_summary = {
                'Democratic': {
                    'yes': 16, 
                    'no': 0}, 
                'Republican': {
                    'yes': 18, 
                    'no': 0}
            }
            # 
            legislator_vote = {'option': 'yes', 'voter_name': 'ACKLEY of Monmouth'}
            test_voter_polarization_helper(legislator_vote, vote_summary, [0.0, 1.0, 1, 1, 1, 1])
            legislator_vote = {'option': 'no', 'voter_name': 'ACKLEY of Monmouth'}
            test_voter_polarization_helper(legislator_vote, vote_summary, [0.0, 0.0, 1, 0, 1, 0])

            vote_summary = {
                'Democratic': {
                    'yes': 3, 
                    'no': 1}, 
                'Republican': {
                    'yes': 0, 
                    'no': 18}
            }
            # 
            legislator_vote = {'option': 'yes', 'voter_name': 'ACKLEY of Monmouth'}
            test_voter_polarization_helper(legislator_vote, vote_summary, [-0.75, 0.75, 1, 1, 1, 0])
            legislator_vote = {'option': 'no', 'voter_name': 'ACKLEY of Monmouth'}
            test_voter_polarization_helper(legislator_vote, vote_summary, [0.75, 1.0, 1, 0, 1, 1])

            vote_summary = {
                'Democratic': {
                    'yes': 1, 
                    'no': 3}, 
                'Republican': {
                    'yes': 18, 
                    'no': 0}
            }
            # 
            legislator_vote = {'option': 'yes', 'voter_name': 'ACKLEY of Monmouth'}
            test_voter_polarization_helper(legislator_vote, vote_summary, [0.75, 1.0, 1, 0, 1, 1])
            legislator_vote = {'option': 'no', 'voter_name': 'ACKLEY of Monmouth'}
            test_voter_polarization_helper(legislator_vote, vote_summary, [-0.75, 0.75, 1, 1, 1, 0])

            # polarization, party_agreement, 
            # democratic_votes, democratic_majority, 
            # republican_votes, republican_majority, 
            vote_summary = {
                'Democratic': {
                    'yes': 3, 
                    'no': 3}, 
                'Republican': {
                    'yes': 18, 
                    'no': 18}
            }
            # 
            legislator_vote = {'option': 'yes', 'voter_name': 'ACKLEY of Monmouth'}
            test_voter_polarization_helper(legislator_vote, vote_summary, [0,0.5, 0, 0])
            legislator_vote = {'option': 'no', 'voter_name': 'ACKLEY of Monmouth'}
            test_voter_polarization_helper(legislator_vote, vote_summary, [0,0.5, 0, 0])

        test_voter_polarization()

        class MinMaxHeap():

            def __init__(self, size=5):
                self.size = size
                self.min_heap = []
                self.max_heap = []

            def __str__(self):
                return f"{self.min_heap} / {self.max_heap}"

            def to_dict(self):
                min_heap = self.min_heap
                min_heap.sort(reverse=True)
                min_heap_out = []
                for key, value in min_heap:
                    min_heap_out.append((-key, value))
                max_heap = self.max_heap
                max_heap.sort(reverse=False)
                max_heap_out = []
                for key, value in max_heap:
                    max_heap_out.append((key, value))
                return {
                    "min_heap": min_heap_out,
                    "max_heap": max_heap_out,
                }

            def push(self, key, value):
                if len(self.max_heap) < self.size:
                    heapq.heappush(self.max_heap, (key, value))
                else:
                    heapq.heappushpop(self.max_heap, (key, value))
                if len(self.min_heap) < self.size:
                    heapq.heappush(self.min_heap, (-key, value))
                else:
                    heapq.heappushpop(self.min_heap, (-key, value))
                # print(f"key/value: {key}/{value}")
                # print(f"min_heap: {self.min_heap}")
                # print(f"max_heap: {self.max_heap}")
                pass

        def test_min_max_heap():
            import random
            test_list = list(range(9, -11, -1))
            ran_list = []
            random.shuffle(test_list)
            min_max_heap = MinMaxHeap(3)
            for number in test_list:
                min_max_heap.push(number, number)
                ran_list.append(number)
            ran_list.sort()
            min_heap = min_max_heap.min_heap
            min_heap.sort()
            assert min_heap == [(8, -8), (9, -9), (10, -10)]
            max_heap = min_max_heap.max_heap
            max_heap.sort()
            assert max_heap == [(7, 7), (8, 8), (9, 9)]
            assert str(min_max_heap) == "[(8, -8), (9, -9), (10, -10)] / [(7, 7), (8, 8), (9, 9)]"
            min_max_heap_dict = min_max_heap.to_dict()
            assert min_max_heap_dict['min_heap'] == [(-10, -10), (-9, -9), (-8, -8)]
            assert min_max_heap_dict['max_heap'] == [(7, 7), (8, 8), (9, 9)]

            min_max_heap = MinMaxHeap(3)
            for i in range(10):
                min_max_heap.push(i,i)
            min_heap = min_max_heap.min_heap
            min_heap.sort()
            print(min_max_heap)
            assert min_heap == [(-2, 2), (-1, 1), (0, 0)]
            max_heap = min_max_heap.max_heap
            max_heap.sort()
            assert max_heap == [(7, 7), (8, 8), (9, 9)]
            assert str(min_max_heap) == "[(-2, 2), (-1, 1), (0, 0)] / [(7, 7), (8, 8), (9, 9)]"
            min_max_heap_dict = min_max_heap.to_dict()
            assert min_max_heap_dict['min_heap'] == [(0, 0), (1, 1), (2, 2)]
            assert min_max_heap_dict['max_heap'] == [(7, 7), (8, 8), (9, 9)]

            min_max_heap = MinMaxHeap(3)
            for i in range(9, -11, -1):
                min_max_heap.push(i,i)
            print(min_max_heap)
            min_heap = min_max_heap.min_heap
            min_heap.sort()
            assert min_heap == [(8, -8), (9, -9), (10, -10)]
            max_heap = min_max_heap.max_heap
            max_heap.sort()
            assert max_heap == [(7, 7), (8, 8), (9, 9)]
            assert str(min_max_heap) == "[(8, -8), (9, -9), (10, -10)] / [(7, 7), (8, 8), (9, 9)]"
            min_max_heap_dict = min_max_heap.to_dict()
            assert min_max_heap_dict['min_heap'] == [(-10, -10), (-9, -9), (-8, -8)]
            assert min_max_heap_dict['max_heap'] == [(7, 7), (8, 8), (9, 9)]

        test_min_max_heap()


[(-2, 2), (-1, 1), (0, 0)] / [(7, 7), (9, 9), (8, 8)]
[(8, -8), (10, -10), (9, -9)] / [(7, 7), (9, 9), (8, 8)]


In [35]:
if True: # lookup of mobile column headers
    mobile_lookup_dict = {
        "Polarization": "Polar-\nization",
        "Vote polarization": "Vote\npolar-\nization",
        "Democratic votes yes/no": "Demo-\ncratic\nvotes\nyes/no",
        "Republican votes yes/no": "Repub-\nlican\nvotes\nyes/no",
        "Democratic agreement": "Demo-\ncratic\nagree-\nment",
        "Republican agreement": "Repub-\nlican\nagree-\nment",
    }
    def column_name_transform(value, mobile=False):
        if not mobile:
            return value
        return mobile_lookup_dict.get(value, value)

In [36]:
if True: # formatting constants
    date_format = "%Y-%m-%d"
    default_column_specs = {
        "Vote polarization": {
            "type": "numeric",
            "format": dash_table.FormatTemplate.percentage(1)},
        "Polarization": {
            "type": "numeric",
            "format": dash_table.FormatTemplate.percentage(1)},
        "Democratic agreement": {
            "type": "numeric",
            "format": dash_table.FormatTemplate.percentage(1)},
        "Republican agreement": {
            "type": "numeric",
            "format": dash_table.FormatTemplate.percentage(1)},
        "Session 128": {
            "type": "numeric",
            "format": dash_table.FormatTemplate.percentage(0)},
        "Session 129": {
            "type": "numeric",
            "format": dash_table.FormatTemplate.percentage(0)},
        "Session 130": {
            "type": "numeric",
            "format": dash_table.FormatTemplate.percentage(0)},
        }
    

In [37]:
if True: # extract vote sponsors and voters from session data
    df = pd.read_csv(session_data_file)
    df["Session #"] = df["Session"].apply(lambda x: x[:3])
    session_lookup = defaultdict(dict)
    for index, row in df.iterrows():
        record = row.to_dict()
        session = record["Session #"]
        for key, value in record.items():
            if key != "Session #":
                session_lookup[session][key] = value
    sessions = []
    session_data = {}
    for path in glob.glob(bills_glob):
        sessions.append(splitall(path)[-2])
    print(sessions)

    for session, path in zip(sessions, glob.glob(bills_glob)):
        with open(path) as _file:
            session_data[session] = json.load(_file)
    sponsors = set()
    voters = set()
    voter_last_names = defaultdict(set)
    vote_options = set()
    # lowercase_voter_lookup = {}

    debug_count = 0
    for session in sessions:
        for bill in session_data[session]:
            if "sponsors" in bill:
                for sponsor in bill["sponsors"]:
                    if "name" in sponsor:
                        sponsors.add(sponsor["name"])
                        # example sponsor: Ryan FECTEAU
                        # sponsors.add(munge_string(sponsor["name"]))
            if "votes" in bill:
                # lines = pprint.pformat(bill).split("\n")
                # # print(f"bill.keys(): {bill.keys()}")
                # if 'title' in bill.keys():
                #     print(f"bill['title']: {bill['title']}")
                # if 'raw_text_url' in bill.keys():
                #     print(f"bill['raw_text_url']: {bill['raw_text_url']}")
                # if 'raw_text' in bill.keys():
                #     print(f"bill['raw_text']: {bill['raw_text']}")
                # # print(f"bill.keys(): {bill.keys()}")
                # # print(lines[0])
                # # for line in lines:
                # #     if line.find("http") > -1:
                # #         print(line)
                # debug_count += 1
                # if debug_count == 6:
                #     sys.exit()
                for bill_vote in bill["votes"]:
                    if "votes" in bill_vote:
                        # pprint.pprint(bill_vote)
                        # sys.exit()
                        
                        for legislator_vote in bill_vote["votes"]:
                            if "option" in legislator_vote:
                                vote_options.add(legislator_vote["option"])
                            if "voter_name" in legislator_vote:
                                # munged_name = munge_string(legislator_vote["voter_name"])
                                munged_name = legislator_vote["voter_name"]
                                # lowercase_voter_lookup[munged_name] = legislator_vote["voter_name"]
                                voters.add(munged_name)
                                # example munged name: ACKLEY of Monmouth
                                try:
                                    name, town = munged_name.split(" of ")
                                    voter_last_names[name.strip()].add(town.strip())
                                except ValueError:
                                    print(f"ERROR: {legislator_vote['voter_name']}")
                                    sys.exit()

    print(sorted(list(sponsors))[:5])

    print(vote_options)

['129', '128', '130']
['ACKLEY', 'ALLEY', 'AUSTIN', 'Aaron FREY', 'Abigail GRIFFIN']
{'no', 'other', 'yes'}


In [38]:
if True: # print voters and duplicate voters
    print(sorted(list(voters))[:4])
    for name, towns in voter_last_names.items():
        # if name.upper() == "DECHANT":
        #     print("\n", name, towns, "\n")
        if len(towns) > 1:
            print(f"{name}: {towns}; ", end="")
    print()

['ACKLEY of Monmouth', 'ALLEY of Beals', 'ANDREWS of Paris', 'ARATA of New Gloucester']
AUSTIN: {'Skowhegan', 'Gray'}; BAILEY: {'Saco', 'York', 'Gorham'}; CARNEY: {'Cape Elizabeth', 'Cumberland'}; DAUGHTRY: {'Cumberland', 'Brunswick'}; DOLLOFF: {'Rumford', 'Milton Township'}; FECTEAU: {'Biddeford', 'Augusta'}; FOLEY: {'York', 'Biddeford', 'Wells'}; HICKMAN: {'Winthrop', 'Kennebec'}; KINNEY: {'Knox', 'Limington'}; LOCKMAN: {'Bradley', 'Amherst'}; MADIGAN: {'Rumford', 'Waterville'}; MARTIN: {'Sinclair', 'Eagle Lake', 'Greene'}; MASON: {'Androscoggin', 'Lisbon'}; MAXMIN: {'Lincoln', 'Nobleboro'}; MILLETT: {'Waterford', 'Cape Elizabeth', 'Cumberland'}; PERRY: {'Calais', 'Bangor'}; PIERCE: {'Dresden', 'Falmouth'}; STEWART: {'Presque Isle', 'Aroostook'}; WARREN: {'Scarborough', 'Hallowell'}; WHITE: {'Mars Hill', 'Waterville', 'Washburn'}; BLACK: {'Wilton', 'Franklin'}; FARRIN: {'Norridgewock', 'Somerset'}; GUERIN: {'Glenburn', 'Penobscot'}; HERBIG: {'Waldo', 'Belfast'}; LAWRENCE: {'York', 'E

In [39]:
if True: # define & test Legislator class
    # goal: lookup legislator by sponsor name or vote name 
    # print

    class Legislators():
        def __init__(
            self, 
            legislators_glob, 
            legislature_metadata,
            names_and_cities=None):

            self.data = {}
            self._lookup = defaultdict(set)
            self._inverse_lookup = defaultdict(set)
            self._preferred_inverse_lookup = {}
            self._party_lookup = {}
            self._seat_lookup = {}
            self._chamber_lookup = {}
            self._metadata = None
            self._allowed_roles = ["upper", "lower"]
            self._formatted_city_lookup = {}

            self.read_metadata(legislature_metadata)

            len_uuid = len("24e3b62c-9257-42fa-b0de-815ed7288641") + 1
            family_name_log = [" "]
            for filename in glob.glob(legislators_glob):
                with open(filename) as _file:
                    data = yaml.safe_load(_file)
                    data["filepath"] = splitall(filename)
                    key = os.path.basename(filename).replace(".yml","")
                    self.data[key] = data
                    if "other_names" in data:
                        for other_name in data["other_names"]:
                            if "name" in other_name:
                                self._lookup[munge_string(other_name["name"])].add(key)
                                family_name = munge_string(other_name["name"])
                                if family_name.find(" OF ") == -1:
                                    family_name = munge_string(other_name["name"]).split(" ")[-1]
                                self._lookup[family_name].add(key)
                    if "family_name" in data or "name" in data:
                        if "family_name" in data:
                            family_name = munge_string(data["family_name"])
                        else:
                            if "name" in data:
                                family_name = (
                                    munge_string(
                                        data["name"], 
                                        upper=False).split(" ")[-1])
                                data["family_name"] = family_name
                                family_name = family_name.upper()
                                family_name_log.append(
                                    f"Assuming name is {family_name} for {key[:-len_uuid]}.")
                        if "family_name" in data and "name" in data:
                            index = data["name"].rfind(data["family_name"])
                            if index > -1:
                                data["first_names"] = data["name"][:index].strip()
                            else:
                                LOG.warning(f"WARNING: {data['family_name']} not found in {data['name']}")
                        else:
                            LOG.warning(f"WARNING: No full name found in:\n{data}")
                        self._lookup[family_name].add(key)
                    else:
                        LOG.warning(f"No family name for {key}.")
                        LOG.warning(data)
            LOG.info(" ".join(family_name_log))
            voters_to_deconflict = set()
            if names_and_cities is not None:
                for name, cities in names_and_cities.items():
                    # if name.upper() == "DECHANT": 
                    #     print(name, cities)
                    if len(cities) == 0:
                        LOG.warning(f"WARNING: {name} is not associated with a city.") 
                    for city in cities:
                        self._formatted_city_lookup[city] = city
                        self._formatted_city_lookup[munge_string(city)] = city                        
                        # if city.upper()[:6] == "BRADLE":
                        #     print(f"YAR name >{name}<")
                        #     print(f"YAR city >{city}<")
                        #     print(f"YAR munged>{munge_string(city)}<")
                        for key in self._lookup[name.upper()]:
                            dict_string = str(self.data[key]).upper()
                            index = dict_string.find(city.upper())
                            # if city.upper()[:6] == "MONMOU":
                            #     print(key)
                            if index > -1:
                                short_key = munge_string(f"{name} of {city}")
                                self._lookup[short_key].add(key)
                                if len(self._lookup[short_key]) > 1:
                                    voters_to_deconflict.add(short_key)
                                # formatted_city = str(self.data[key])[index:index+len(city)]

                                # print(f">{formatted_city}< >{city}< >{munge_string(city)}<")
                deconflict_log_message = [" "]
                for short_key in voters_to_deconflict:
                    legislature_match = False
                    for duplicate_key in self._lookup[short_key]:
                        data = self.data[duplicate_key]
                        if 'legislature' in data["filepath"]:
                            legislature_match = True
                    keys_to_remove = set()
                    for duplicate_key in self._lookup[short_key]:
                        data = self.data[duplicate_key]
                        if 'retired' in data["filepath"]:
                            if legislature_match:
                                keys_to_remove.add(duplicate_key)
                    for duplicate_key in keys_to_remove:
                        self._lookup[short_key].remove(duplicate_key)
                        deconflict_log_message.append(
                            f"Removed duplicate 'retired' record for {short_key}.")
                    if len(self._lookup[short_key]) > 1:
                        for duplicate_key in self._lookup[short_key]:
                            data = self.data[duplicate_key]
                            print(f"{data['filepath']}")
                            if "party" in data:
                                #  and "name" in data["party"]:
                                print(data["party"])
                LOG.info(" ".join(deconflict_log_message))

        def print_log(self, message):
            print(message)

        def read_metadata(self, legislature_metadata):
            with open(legislature_metadata) as _file:
                data = yaml.safe_load(_file)
            self._legislative_metadata = data

        def _find_match(self, key):
            if (key in self._lookup 
                and len(self._lookup[key]) == 1):
                return self.data[next(iter(self._lookup[key]))]
            else:
                return None

        def lookup(self, string):
            key = munge_string(string)
            match = self._find_match(key)
            if match:
                return match
            of_index = key.find(" OF ")
            new_key = None
            if of_index > -1:
                new_key = key[:of_index]
                # print(f"{key}: >{new_key}<")
                match = self._find_match(new_key)
                if match:
                    self._lookup[key].add(
                        next(iter(self._lookup[new_key])))
                    return match
            if (len(self._lookup[key]) > 1 
                or (new_key and len(self._lookup[new_key]) > 1)):
                raise ValueError(f"Multiple matches found for {key}")
            else:
                raise ValueError(f"No match found for {key}")

            # long_keys = self._lookup[key]
            # if len(long_keys) == 0:
            #     raise ValueError(f"No match found for {key}")
            # if len(long_keys) > 1:
            # else:
            #     for long_key in long_keys:
            #         print(f"{long_key}")
            #         return self.data[long_key]

        def formatted_name(self, string, type="name_and_city"):
            allowed_types = ["name_and_city", "name_and_initial", "full_name_and_city"] 
            if type not in allowed_types:
                raise ValueError(f"{type} not in {allowed_types}")
            key = munge_string(string)
            data = self.lookup(string)
            if "_formatted_name" in data:
                return data["_formatted_name"][type]
            city = self.lookup_city(string)
            if city == "NA":
                city_suffix = ""
            else:
                city_suffix = f" of {city}"
            if ("first_names" in data
                and len(data["first_names"]) > 0):
                initial = data["first_names"][:1].upper()
                prefix = f"{initial}. "
                initial_suffix = f", {initial}."
                full_name = f"{data['first_names']} {data['family_name']}"
            else:
                prefix = ""
                initial_suffix = ""
                first_names = data['family_name']
                full_name = data['name']
            data["_formatted_name"] = {
                "name_and_city": (
                    f"{prefix}{data['family_name']}{city_suffix}"),
                "name_and_initial": (
                    f"{data['family_name']}{initial_suffix}"),
                "full_name_and_city": (
                    f"{full_name}{city_suffix}"
                )
            }
            return data["_formatted_name"][type]

        def lookup_city(self, string):
            key = munge_string(string)
            data = self.lookup(string)
            if "_formatted_city" in data:
                return data["_formatted_city"]
            index = key.find(" OF ")
            if index == -1:
                data["_formatted_city"] = "NA"
                return data["_formatted_city"]
            else:
                index += len(" OF ")
                data["_formatted_city"] = (
                    self._formatted_city_lookup[string[index:]])
                return data["_formatted_city"]

        def lookup_party(self, string):
            key = munge_string(string)
            if key in self._party_lookup:
                return self._party_lookup[key]
            result = ""
            data = self.lookup(key)
            # if data:
            #     print(f"match! {len(data)}")
            if data and "party" in data:
                sorted_parties = set()
                for party in data["party"]:
                    if "name" in party:
                        sorted_parties.add(party["name"])
                result = "/".join(sorted(list(sorted_parties)))
            if not result:
                result = "NA"
            self._party_lookup[key] = result
            return result

        def lookup_seat(self, string):
            key = munge_string(string)
            if key in self._seat_lookup:
                return self._seat_lookup[key]            
            # add district data to legislator table (upper:Senate/lower:House; S/H) roles/type/district
            result = ""
            data = self.lookup(key)
            if not "roles" in data:
                raise ValueError(f"No 'roles' data for {key}")
            seats = []
            for role in data["roles"]:
                if role["type"] in self._allowed_roles:
                    chamber = self._legislative_metadata[role["type"]]["abbreviation"]
                    try:
                        district = role["district"]
                    except KeyError:
                        LOG.warning(f"No district found for {key}")
                        district = "(NA)"
                    seats.append(f"{chamber}-{district}")
            self._seat_lookup[key] = "/".join(sorted(seats))
            return self._seat_lookup[key]

        def build_inverse_lookup(self):
            for key, values in self._lookup.items():
                for value in values:
                    self._inverse_lookup[value].add(key)
            for key, values in self._inverse_lookup.items():
                new_set = set()
                for value in values:
                    if value.find(" OF ") > -1:
                        new_set.add(value)
                if new_set:
                    self._inverse_lookup[key] = new_set

        def inverse_lookup(self, value):
            if value in self._preferred_inverse_lookup:
                return self._preferred_inverse_lookup[value]
            try:
                result = self._inverse_lookup[value]
                if len(result) == 1:
                    return next(iter(result))
                else:
                    return result
            except KeyError as ex:
                print(f"KeyError: {ex}")
                return None

    legislators = Legislators(legislators_glob, legislature_metadata, voter_last_names)

    parties = set()
    success = 0
    failure = 0
    # want to find duplicate (e.g. Black of Franklin/Wilton)
    record_to_name_map = defaultdict(set)
    for name, cities in voter_last_names.items():
        # if name.find(" ") > -1:
        #     print(f"name with space >{name}< >{cities}<")
        for city in cities:
            key = munge_string(f"{name} of {city}")
            try:
                # if city.upper()[:6] == "YARMOU":
                #     print(key, name, city)
                #     pprint.pprint(legislators.lookup(key))
                #     print(legislators.lookup_city(key))
                #     print(legislators.formatted_name(key))
                #     pprint.pprint(legislators.lookup(key))
                    # pprint.pprint(legislators._formatted_city_lookup)
                # sys.exit()
                record = legislators.lookup(key)
                record_to_name_map[record["id"]].add(key)
                parties.add(legislators.lookup_party(key))
                success += 1
            except ValueError:
                if failure == 0:
                    print()
                print(f"Failure during lookup of {key}")
                if name in legislators._lookup:
                    print(legislators._lookup[name])
                # print(voter_last_names[munge_string(name)])
                failure += 1
    duplicate_voter_keys = {}
    for id, keys in record_to_name_map.items():
        if len(keys) > 1:
            print(f"duplicate voter keys: {keys}")
        for key in keys:
            duplicate_voter_keys[key] = keys
    if failure == 0:
        LOG.info(f"lookup for all voting legislatures ({success}) is successful!")
    else:
        LOG.warning(f"errors during lookup for all voting legislatures. {success} matches; {failure} failures.")
    LOG.info(f"party list: {parties}")
    # print(legislators.formatted_name("BAILEY OF SACO"))
    # Douglas-K-Damon-6179a243-8b05-471b-9b00-9ec8aeda6858-R
    #    Donna-Bailey-d071215f-4839-4c4c-ab4d-f3c7179004a6
    # print(legislators.lookup("BAILEY OF SACO"))
    legislators.build_inverse_lookup()
    print(legislators._lookup["BAILEY OF SACO"])
    # pprint.pprint(legislators._inverse_lookup)
    print(legislators.inverse_lookup("Donna-Bailey-d071215f-4839-4c4c-ab4d-f3c7179004a6"))

            #     self._lookup = defaultdict(set)
            # self._inverse_lookup = {}


INFO:__main__:  Assuming name is SWALLOW for Gregory-Lewis-Swallow. Assuming name is DENK for Diane-M-Denk. Assuming name is INGWERSEN for Henry-L-Ingwersen. Assuming name is PEOPLES for Ann-E-Peoples. Assuming name is VEROW for Arthur-C-Verow. Assuming name is BABINE for Shawn-A-Babine. Assuming name is KESCHL for Dennis-L-Keschl. Assuming name is CURTIS for Philip-A-Curtis. Assuming name is HOBBS for Daniel-J-Hobbs. Assuming name is LAROCHELLE for Raegan-French-LaRochelle.
INFO:__main__:  Removed duplicate 'retired' record for SANBORN OF CUMBERLAND. Removed duplicate 'retired' record for FARRIN OF SOMERSET. Removed duplicate 'retired' record for FARRIN OF NORRIDGEWOCK. Removed duplicate 'retired' record for NADEAU OF WINSLOW. Removed duplicate 'retired' record for HICKMAN OF WINTHROP. Removed duplicate 'retired' record for POULIOT OF AUGUSTA. Removed duplicate 'retired' record for SANBORN OF PORTLAND. Removed duplicate 'retired' record for LAWRENCE OF ELIOT. Removed duplicate 'retire

duplicate voter keys: {'BAILEY OF SACO', 'BAILEY OF YORK'}
duplicate voter keys: {'CARNEY OF CAPE ELIZABETH', 'CARNEY OF CUMBERLAND'}
duplicate voter keys: {'DAUGHTRY OF BRUNSWICK', 'DAUGHTRY OF CUMBERLAND'}
duplicate voter keys: {'DOLLOFF OF MILTON TOWNSHIP', 'DOLLOFF OF RUMFORD'}
duplicate voter keys: {'FECTEAU OF AUGUSTA', 'FECTEAU OF BIDDEFORD'}
duplicate voter keys: {'FOLEY OF YORK', 'FOLEY OF WELLS'}
duplicate voter keys: {'HICKMAN OF WINTHROP', 'HICKMAN OF KENNEBEC'}
duplicate voter keys: {'LOCKMAN OF BRADLEY', 'LOCKMAN OF AMHERST'}
duplicate voter keys: {'MAXMIN OF NOBLEBORO', 'MAXMIN OF LINCOLN'}
duplicate voter keys: {'MILLETT OF CAPE ELIZABETH', 'MILLETT OF CUMBERLAND'}
duplicate voter keys: {'STEWART OF PRESQUE ISLE', 'STEWART OF AROOSTOOK'}
duplicate voter keys: {'WHITE OF WASHBURN', 'WHITE OF MARS HILL'}
duplicate voter keys: {'BLACK OF FRANKLIN', 'BLACK OF WILTON'}
duplicate voter keys: {'FARRIN OF SOMERSET', 'FARRIN OF NORRIDGEWOCK'}
duplicate voter keys: {'GUERIN OF PE

In [40]:
if True: # process votes for party lineness
    # use set
    date_format = "%Y-%m-%d"
    votes_dict = {}
    vote_ratios = defaultdict(list)
    party_line_dict = defaultdict(int)
    party_line_dict_0_7 = defaultdict(int)
    # Vote
    # Vote polarization
    # Democratic votes yes/no
    # Republican votes yes/no
    # Title
    # Date
    vote_polarization_columns = [
        "Party", 
        "Polarization", 
        "Unanimity",
        "Yes polarization",
        "Republican yes votes",
        "Republican no votes",
        "Democratic yes votes",
        "Democratic no votes",
        "Republican yes",
        "Democratic yes",
        "Party polarization",
        "Title",
        "title_w_breaks",
        "Date",
        "Session",
        "Key",
        ]
    vote_polarization_data = []
    # for session_index, session in enumerate(sessions):
        # print(len(sessions), session_index)
    for session in sessions:
        for bill in session_data[session]:
            if "votes" in bill:
                key = str(uuid.uuid4())
                date_order_error = False
                last_date = None
                vote_polarization_rows = []
                for bill_vote in bill["votes"]:
                    if "votes" in bill_vote:
                        if ("start_date" in bill_vote):
                            if not last_date:
                                last_date = (
                                    datetime.datetime.strptime(
                                        bill_vote["start_date"],
                                        "%Y-%m-%d"))
                            else:
                                next_date = (
                                    datetime.datetime.strptime(
                                        bill_vote["start_date"],
                                        "%Y-%m-%d"))
                                if next_date < last_date:
                                    raise ValueError(
                                        f"WARNING: date order for {key} is out of order. "
                                        f"({next_date} < {last_date}")
                                else:
                                    last_date = next_date  
                        # pprint.pprint(bill)
                        # sys.exit()
                        votes_dict[key] = {
                            "session_number": session,
                            "bill": bill, 
                            "bill_vote": bill_vote,
                            "vote_date": bill_vote["start_date"],
                            "vote_summary": defaultdict(lambda: defaultdict(int)),
                            "party_line_result": None,
                            "party_line_result_0_7": None,
                            "final_vote": False,
                        }
                        for legislator_vote in bill_vote["votes"]:
                            if "voter_name" in legislator_vote:
                                voter = legislator_vote["voter_name"]
                                voters.add(voter)
                                if "option" in legislator_vote:
                                    votes_dict[key][
                                        "vote_summary"][
                                            legislators.lookup_party(voter)][
                                                legislator_vote["option"]] += 1
                        for party, votes in votes_dict[key]["vote_summary"].items():
                            yes = votes.get("yes", 0)
                            no = votes.get("no", 0)
                            if yes > 0 or no > 0:
                                vote_ratios[party].append(
                                    yes/(yes + no))
                        votes_summary = votes_dict[key]["vote_summary"]
                        yes_polarization_metric = yes_polarization(votes_summary)
                        if "Democratic" in votes_summary:
                            try:
                                polarization = (
                                    polarization_metric_2(votes_summary, "Democratic", "Republican"))
                                
                                row = [
                                    "Democratic",
                                    -polarization[0],
                                    polarization[1],
                                    yes_polarization_metric,
                                    votes_summary["Republican"].get("yes", 0),
                                    votes_summary["Republican"].get("no", 0),
                                    votes_summary["Democratic"].get("yes", 0),
                                    votes_summary["Democratic"].get("no", 0),
                                    polarization[3],
                                    polarization[2],
                                    -polarization[4],
                                    bill["title"].replace("'S", "'s"),
                                    textwrap.fill(
                                        bill["title"], width=30).replace(
                                            "\n","<br>").replace(
                                                "'S", "'s"),
                                    last_date,
                                    session,
                                    str(key),
                                ]
                                # if len(textwrap.fill(bill["title"], width=30)) > 30:
                                #     print(textwrap.fill(bill["title"], width=30))
                                #     sys.exit()
                            except KeyError:
                                pass
                            vote_polarization_rows.append(row)
                        if "Republican" in votes_summary:
                            try:
                                polarization = (
                                    polarization_metric_2(votes_summary, "Republican", "Democratic"))
                                    
                                row = [
                                    "Republican",
                                    polarization[0],
                                    polarization[1],
                                    yes_polarization_metric,
                                    votes_summary["Republican"].get("yes", 0),
                                    votes_summary["Republican"].get("no", 0),
                                    votes_summary["Democratic"].get("yes", 0),
                                    votes_summary["Democratic"].get("no", 0),
                                    polarization[2],
                                    polarization[3],
                                    polarization[4],
                                    bill["title"].replace("'S", "'s"),
                                    textwrap.fill(
                                        bill["title"], width=30).replace(
                                            "\n","<br>").replace(
                                                "'S", "'s"),
                                    last_date,
                                    session,
                                    str(key),
                                ]
                            except KeyError:
                                pass
                            vote_polarization_rows.append(row)

                        # print(vote_polarization_dict)
                        # sys.exit()
                        party_line_result = party_line_vote_test(votes_dict[key]["vote_summary"])
                        party_line_result_0_7 = party_line_vote_test(
                            votes_dict[key]["vote_summary"], threshold=0.7)
                        votes_dict[key]["party_line_result"] = party_line_result
                        votes_dict[key]["party_line_result_0_7"] = party_line_result_0_7
                        party_line_dict[party_line_result] += 1 
                        party_line_dict_0_7[party_line_result_0_7] += 1
                if key in votes_dict:
                    votes_dict[key]["final_vote"] = True
                    vote_polarization_data.extend(vote_polarization_rows[-2:])
    # pprint.pprint(votes_dict[key]["vote_summary"])
    pprint.pprint(party_line_dict)
    pprint.pprint(party_line_dict_0_7)
    vote_polarization_df = pd.DataFrame(
        data=vote_polarization_data, columns=vote_polarization_columns)
    vote_polarization_df["Democratic votes yes/no"] = vote_polarization_df.apply(
        lambda row: f'{row["Democratic yes votes"]}/{row["Democratic no votes"]}', axis=1)
    vote_polarization_df["Republican votes yes/no"] = vote_polarization_df.apply(
        lambda row: f'{row["Republican yes votes"]}/{row["Republican no votes"]}', axis=1)
    vote_polarization_df = vote_polarization_df.sort_values(by=['Yes polarization'])
    vote_polarization_df = vote_polarization_df.sort_values(by=['Session'], ascending=False)
    # Democratic votes yes/no
    # Republican votes yes/no
    # Title
    # Date
    # vote_polarization_columns = [
    #     "Party", 
    #     "Polarization", 
    #     "Unanimity",
    #     "Yes polarization",
    #     "Republican yes votes",
    #     "Republican no votes",
    #     "Democratic yes votes",

    def update_row(row):
        new_row.pop("Polarization")
        new_row["Date"] = new_row["Date"].strftime(date_format)
        new_row["Title"] = new_row["Title"].replace("<br>"," ").replace("'S", "'s")
        new_row["Democratic votes yes/no"] = (
            f'{new_row.pop("Democratic yes votes")}/'
            f'{new_row.pop("Democratic no votes")}')
        new_row["Republican votes yes/no"] = (
            f'{new_row.pop("Republican yes votes")}/'
            f'{new_row.pop("Republican no votes")}')

    bill_lookup = {}
    for _, row in vote_polarization_df.iterrows():
        new_row = row.to_dict()
        update_row(new_row)
        bill_lookup[row["Key"]] = new_row
    export_dict("bill_lookup")
    display(vote_polarization_df[:2])
    display(vote_polarization_df[-2:])
    print(f"len(votes_dict)*2: {len(votes_dict)*2}")
    # Straight Party-Line: All Republicans took one position, all Democrats took the other.
    # Near Party-Line: At least 90% of one or both parties voted the same way.
    # Not Party-Line: Neither party had at least 90% of members vote the same way.

    # # party_line_dict = defaultdict(int)
    # for value in votes_dict.values():
    #     print(value["vote_summary"])
    #     sys.exit()
    ic(len(vote_polarization_df))
    ic(vote_polarization_df["Party"].unique())
    # print(len(votes_dict["vote_summary"]))
    vote_polarization_df_copy = vote_polarization_df.copy()
    pass


defaultdict(<class 'int'>,
            {'bipartisan': 367,
             'near_party_line': 2172,
             'not_party_line': 125,
             'straight_party_line': 416})
defaultdict(<class 'int'>,
            {'bipartisan': 548,
             'near_party_line': 2099,
             'not_party_line': 17,
             'straight_party_line': 416})


,Party,Polarization,Unanimity,Yes polarization,Republican yes votes,Republican no votes,Democratic yes votes,Democratic no votes,Republican yes,Democratic yes,Party polarization,Title,title_w_breaks,Date,Session,Key,Democratic votes yes/no,Republican votes yes/no
3077,Republican,1.000000,1.0,-1.000000,0,12,18,0,0.0,1.000000,-1.000000,"Resolve, To Establish A Pilot Program To Encou...","Resolve, To Establish A Pilot<br>Program To En...",2022-03-09,130,2772e24a-9d5d-4edb-a2d3-5e99ed340509,18/0,0/12
2599,Republican,0.893333,1.0,-0.893333,0,62,67,8,0.0,0.893333,-0.893333,An Act To Amend The Maine Bail Code Regarding ...,An Act To Amend The Maine Bail<br>Code Regardi...,2021-06-30,130,1a355a89-f3fd-4a32-a055-bee63c8eedc9,67/8,0/62


,Party,Polarization,Unanimity,Yes polarization,Republican yes votes,Republican no votes,Democratic yes votes,Democratic no votes,Republican yes,Democratic yes,Party polarization,Title,title_w_breaks,Date,Session,Key,Democratic votes yes/no,Republican votes yes/no
1675,Republican,0.591348,0.985714,-0.591348,1,69,43,28,0.014286,0.605634,-0.591348,An Act To Provide For A Later Starting Time Fo...,An Act To Provide For A Later<br>Starting Time...,2017-05-11,128,a87218d6-9b4a-4844-9f68-7c4306237ce1,43/28,1/69
1627,Republican,1.000000,1.000000,1.000000,18,0,0,17,1.000000,0.000000,1.000000,An Act To Base The Minimum Wage On A New Engla...,An Act To Base The Minimum<br>Wage On A New En...,2017-06-16,128,e9ffc0f8-98b7-45d8-a7a9-fae7c881146d,0/17,18/0


ic| len(vote_polarization_df): 3078
ic| vote_polarization_df["Party"].unique(): array(['Republican', 'Democratic'], dtype=object)


len(votes_dict)*2: 3078


In [41]:
if True: # analyze voters
    # Straight Party-Line: All Republicans took one position, all Democrats took the other.
    # Near Party-Line: At least 90% of one or both parties voted the same way.
    # Not Party-Line: Neither party had at least 90% of members vote the same way.
    # Bipartisan: at least 90% of both parties voted the same way
    
    # voter_result: bipartisan, party_line_left; party_line_right; not_party_line
    # {'near_party_line': 904, 'not_party_line': 67, 'straight_party_line': 79, 'bipartisan': 232}
    # print

    # votes_dict = {}
    # party_line_dict = defaultdict(int)
    voter_dict_0_9 = defaultdict(lambda: defaultdict(int))
    voter_dict_0_7 = defaultdict(lambda: defaultdict(int))
    session_dict_0_9 = defaultdict(lambda: defaultdict(int))
    session_dict_0_7 = defaultdict(lambda: defaultdict(int))

    # session_dict_0_9: session: not, near, straight, bi
    voter_polarization_columns = [
        "Key",
        "Name",
        "Party", 
        "Seat", 
        "Polarization", 
        "Party agreement",
        "Democratic agreement",
        "Republican agreement",
        ]
    voter_polarization_data = []
    voter_polarization_dict = defaultdict(lambda: defaultdict(float))
    voter_bills_heap = {} # (polarization, bill_id)
    voter_bills_dict = defaultdict(lambda: defaultdict(str)) # (polarization, bill_id)

    for key, record in votes_dict.items():
        # if "votes" in bill_vote:
        #     key = uuid.uuid4()
        #     votes_dict[key] = {
        #         "session_index": session_index,
        #         "bill": bill, 
        #         "bill_vote": bill_vote,
        #         "vote_date": bill_vote["start_date"],
        #         "vote_summary": defaultdict(lambda: defaultdict(int)),
        #     }
        # party_line_result
        session_dict_0_9[record["session_number"]][record["party_line_result"]] += 1
        session_dict_0_7[record["session_number"]][record["party_line_result_0_7"]] += 1
        if record["final_vote"]:
            for legislator_vote in record["bill_vote"]["votes"]:
                # pprint.pprint(record.keys())
                # pprint.pprint(["vote_summary\n", record["vote_summary"]])
                # pprint.pprint(["legislator_vote\n", legislator_vote])
                # sys.exit()
                voter_dict_0_9[
                    munge_string(
                        legislator_vote["voter_name"])][
                            party_line_voter_test(
                                legislator_vote, 
                                record["vote_summary"], 
                                threshold=0.9)] += 1
                voter_dict_0_7[
                    munge_string(
                        legislator_vote["voter_name"])][
                            party_line_voter_test(
                                legislator_vote, 
                                record["vote_summary"], 
                                threshold=0.7)] += 1
                try:
                    this_voter_polarization = voter_polarization(
                        legislator_vote,
                        record["vote_summary"]
                    )
                    voter_key = munge_string(legislator_vote["voter_name"])
                    voter_keys = duplicate_voter_keys[voter_key]
                    for voter_key in voter_keys:
                        if voter_key not in voter_bills_heap:
                            voter_bills_heap[voter_key] = MinMaxHeap(5)
                        voter_bills_heap[voter_key].push(this_voter_polarization[0], key)
                        voter_bills_dict[voter_key][key] = legislator_vote["option"]
                        voter_polarization_dict[munge_string(voter_key)][
                            "Polarization"] += this_voter_polarization[0]
                        voter_polarization_dict[munge_string(voter_key)][
                            "party_agreement"] += this_voter_polarization[1]
                        voter_polarization_dict[munge_string(voter_key)][
                            "count"] += 1
                        voter_polarization_dict[munge_string(voter_key)][
                            "democratic_votes"] += this_voter_polarization[2]
                        voter_polarization_dict[munge_string(voter_key)][
                            "democratic_majority"] += this_voter_polarization[3]
                        voter_polarization_dict[munge_string(voter_key)][
                            "republican_votes"] += this_voter_polarization[4]
                        voter_polarization_dict[munge_string(voter_key)][
                            "republican_majority"] += this_voter_polarization[5]
                except ValueError:
                    pass
                #     if "voter_name" in le  #         voter = voter_key
                #         voters.add(voter)
                #         if "option" in legislator_vote:
                #             votes_dict[key][
                #                 "vote_summary"][
                #                     legislators.lookup_party(voter)][
                #                         legislator_vote["option"]] += 1
                # party_line_result = party_line_test(votes_dict[key]["vote_summary"])
                # votes_dict[key]["party_line_result"] = party_line_result
                # party_line_dict[party_line_result] += 1  
            # pprint.pprint(voter_dict)
            # sys.exit()
            pass
    # pprint.pprint(session_dict_0_9)
    # pprint.pprint(["last result", ['ACKLEY OF MONMOUTH',
    #          {'bipartisan': 100,
    #           'not_party_line': 168,
    #           'party_line_left': 785,
    #           'party_line_right': 58}]])
    # pprint.pprint(voter_polarization_dict)
    for key, record in voter_polarization_dict.items():
        democratic_agreement = record["democratic_majority"]/record["democratic_votes"]
        republican_agreement = record["republican_majority"]/record["republican_votes"]
        if record["Polarization"]/record["count"] < 0:
            party_agreement = democratic_agreement
        else:
            party_agreement = republican_agreement
        # if key[:5] == "BLACK":
        #     print(key)
        #     pprint.pprint(record)
        row = [
            key,
            key,
            legislators.lookup_party(key),
            legislators.lookup_seat(key),
            record["Polarization"]/record["count"],
            party_agreement,
            democratic_agreement,
            republican_agreement,
        ]
        voter_polarization_data.append(row)
    voter_polarization_df = pd.DataFrame(
        data=voter_polarization_data,
        columns=voter_polarization_columns)
    voter_polarization_df["Party"] = (
        voter_polarization_df.apply (
            lambda row: row["Party"].replace("/", "/<br>"), axis=1))
    # display(voter_polarization_df[
    #     voter_polarization_df["Key"] == "BLACK OF FRANKLIN"])
    display(voter_polarization_df[:2])
    display(voter_polarization_df[-2:])
    # print(voter_bills_heap['ACKLEY OF MONMOUTH'])
    voter_bills_heap_export = {}
    # count = 0
    for voter_key, heap in voter_bills_heap.items():
        # debug
        # print(voter_key)
        # display(voter_polarization_df[
        #     voter_polarization_df["voter_Key"] == voter_key])
        row = voter_polarization_df[voter_polarization_df["Key"] == voter_key]
        # print(f"row.index: {row.index}")
        index = row.index[0]
        party_lean = get_party_lean(row.at[index,'Party'], row.at[index, 'Polarization'])
        # print(f"party_lean: {party_lean}")
        heap_dict = heap.to_dict()
        if party_lean == "Democratic":
            heap_name = "max_heap"
            compare_sign = 1
        if party_lean == "Republican":
            heap_name = "min_heap"
            compare_sign = -1
        new_list = []
        initial_length = len(heap_dict[heap_name])
        for key, value in heap_dict[heap_name]:
            # print(key, value, type(key), type(value))
            if key * compare_sign > 0:
                new_list.append([key, value])
        heap_dict[heap_name] = new_list
        if len(new_list) < initial_length:
            print(f"{voter_key} ({len(new_list)}: {new_list}")
        # count += 1
        # if count == 2:
        #     sys.exit()
        voter_bills_heap_export[voter_key] = heap_dict

    export_dict("voter_bills_heap_export")
    export_dict("voter_bills_dict")
    
    # for key, value in voter_dict_0_9.items():
    #     if key == 'ACKLEY OF MONMOUTH':
    #         pprint.pprint([key, value])
    #         print(legislators.lookup_seat("ACKLEY OF MONMOUTH"))
    pass

,Key,Name,Party,Seat,Polarization,Party agreement,Democratic agreement,Republican agreement
0,BELLOWS OF KENNEBEC,BELLOWS OF KENNEBEC,Democratic,S-14,-0.531732,0.943694,0.943694,0.365909
1,BLACK OF FRANKLIN,BLACK OF FRANKLIN,Republican,H-114/S-17,0.620754,0.956466,0.273189,0.956466


,Key,Name,Party,Seat,Polarization,Party agreement,Democratic agreement,Republican agreement
317,LAROCHELLE OF AUGUSTA,LAROCHELLE OF AUGUSTA,Democratic,H-86,-0.746854,1.000000,1.000000,0.197183
318,BRENNER OF CUMBERLAND,BRENNER OF CUMBERLAND,Democratic,S-30,-0.675205,0.982394,0.982394,0.249110


SANBORN L OF CUMBERLAND (3: [[0.1428571428571429, '54ae1ea0-5758-4bfb-b20d-eb06003b74bd'], [0.2857142857142857, 'c8755d5b-a2eb-41a6-b87f-475882205397'], [0.47619047619047616, '6953136b-2d9a-4a04-8aeb-db97605a8307']]
BELL OF YARMOUTH (3: [[0.07792207792207795, 'afa34caf-208a-43b4-81bf-ec25712df809'], [0.19685726756874722, '7e3e9ca4-995e-46ea-9d4f-d40440afe865'], [0.34300919206579583, 'beefdba5-32f3-452c-8c8e-30140455c931']]
LOOKNER OF PORTLAND (4: [[0.07792207792207795, 'afa34caf-208a-43b4-81bf-ec25712df809'], [0.19685726756874722, '7e3e9ca4-995e-46ea-9d4f-d40440afe865'], [0.30288355712084525, '3142f729-afd3-4f40-87c1-0fb462660a1d'], [0.858974358974359, 'e301a5e1-348d-496c-b34b-62bc16b2365c']]
OSHER OF ORONO (4: [[0.07792207792207795, 'afa34caf-208a-43b4-81bf-ec25712df809'], [0.19685726756874722, '7e3e9ca4-995e-46ea-9d4f-d40440afe865'], [0.5063291139240507, '98c2a7cc-af3e-444a-8f30-acaa0f25b962'], [0.6351351351351351, '5a840771-2c2d-44c6-8fc3-b2a7778146c7']]
LAROCHELLE OF AUGUSTA (2: [[

In [42]:
if True: # second ternary
    # every vote: ratio left; ratio right
    # 
    state = "Maine"
    display(vote_polarization_df[:3])
    # print(vote_polarization_df.columns)
    # sys.exit()
    # fig = px.scatter(
    #     vote_polarization_df,
    #     x="Polarization", 
    #     y = "Unanimity",
    #     color="Party",
    #     symbol="Session",
    #     hover_data=vote_polarization_columns
    # )
    # Index(['Party', 'Polarization', 'Unanimity', 'Yes polarization',
    #     'Republican yes votes', 'Republican no votes', 'Democratic yes votes',
    #     'Democratic no votes', 'Republican yes', 'Democratic yes', 'Title',
    #     'Date', 'Session', 'Key']
    fig = px.scatter(
        vote_polarization_df[vote_polarization_df["Party"] == "Democratic"],
        x="Republican yes", 
        y = "Democratic yes",
        symbol="Session",
        color="Party polarization",
        color_continuous_scale=["blue", "red"],
        hover_data=vote_polarization_columns,
    )
    print(vote_polarization_columns)
    fig.update_layout(legend_orientation="h")

    # fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=0,
    #                                       ticks="outside",
    #                                     #   ticksuffix=" bills",
    #                                       ))
    fig.show()
    vote_polarization_df = vote_polarization_df[vote_polarization_df["Party"] == "Democratic"]
    export_df("vote_polarization_df")

    print(voter_polarization_columns)

    voter_polarization_df["Name"] = (
        voter_polarization_df.apply (lambda row: legislators.formatted_name(row['Key']), axis=1))
    fig = px.scatter(
        voter_polarization_df,
        x="Polarization", 
        y = "Party agreement",
        color="Party",
        # hover_data=voter_polarization_columns,
        hover_data = {
            'Name': True,
            'Party': True,
            'Seat': True,
            'Polarization': ":,.0%",
            'Party agreement': ":,.0%",
        },
        title=f"{state} Legislator Polarization. Democratic/left; Republican/right.",
    )
    fig.update_layout(yaxis_tickformat = ',.0%')
    fig.update_layout(xaxis_tickformat = ',.0%')
    fig.show()
    export_df("voter_polarization_df")
    pass

,Party,Polarization,Unanimity,Yes polarization,Republican yes votes,Republican no votes,Democratic yes votes,Democratic no votes,Republican yes,Democratic yes,Party polarization,Title,title_w_breaks,Date,Session,Key,Democratic votes yes/no,Republican votes yes/no
3077,Republican,1.000000,1.0,-1.000000,0,12,18,0,0.0,1.000000,-1.000000,"Resolve, To Establish A Pilot Program To Encou...","Resolve, To Establish A Pilot<br>Program To En...",2022-03-09,130,2772e24a-9d5d-4edb-a2d3-5e99ed340509,18/0,0/12
2599,Republican,0.893333,1.0,-0.893333,0,62,67,8,0.0,0.893333,-0.893333,An Act To Amend The Maine Bail Code Regarding ...,An Act To Amend The Maine Bail<br>Code Regardi...,2021-06-30,130,1a355a89-f3fd-4a32-a055-bee63c8eedc9,67/8,0/62
2975,Republican,0.888889,1.0,-0.888889,0,11,16,2,0.0,0.888889,-0.888889,An Act Implementing The Recommendations Of The...,An Act Implementing The<br>Recommendations Of ...,2022-04-25,130,001d68e6-c434-4b52-a2d4-b3856136f36e,16/2,0/11


['Party', 'Polarization', 'Unanimity', 'Yes polarization', 'Republican yes votes', 'Republican no votes', 'Democratic yes votes', 'Democratic no votes', 'Republican yes', 'Democratic yes', 'Party polarization', 'Title', 'title_w_breaks', 'Date', 'Session', 'Key']


['Key', 'Name', 'Party', 'Seat', 'Polarization', 'Party agreement', 'Democratic agreement', 'Republican agreement']


In [43]:
if True: # voter_polarization_highlights
    voter_polarization_df["abs polarization"] = (
        voter_polarization_df.apply (lambda row: abs(row["Polarization"]), axis=1))
    voter_polarization_df = voter_polarization_df.sort_values(["Polarization"])
    a = voter_polarization_df[:1]
    b = voter_polarization_df[-1:]
    voter_polarization_df = voter_polarization_df.sort_values(["Democratic agreement"])
    c = voter_polarization_df[-1:]
    c2 = voter_polarization_df[voter_polarization_df["Party"] == "Republican"][-2:]
    voter_polarization_df = voter_polarization_df.sort_values(["Republican agreement"])
    d = voter_polarization_df[-1:]
    d2 = voter_polarization_df[voter_polarization_df["Party"] == "Democratic"][-1:]
    voter_polarization_df = voter_polarization_df.sort_values(["Party agreement"])
    e = voter_polarization_df[:1]
    voter_polarization_df = voter_polarization_df.sort_values(["abs polarization"])
    f = voter_polarization_df[:1]
    voter_polarization_highlights_df = pd.concat([a, b, c, c2, d, d2, e, f])
    voter_polarization_highlights_df = voter_polarization_highlights_df.sort_values(["Polarization"])
    voter_polarization_highlights_df = voter_polarization_highlights_df.drop_duplicates()
    voter_polarization_highlights_df["Name"] = (
        voter_polarization_highlights_df.apply (lambda row: legislators.formatted_name(row['Key']), axis=1))
    # voter_polarization_highlights_df["Polarization"] = (
    #     voter_polarization_highlights_df.apply (lambda row: f"{row['Polarization']:.1%}", axis=1))
    # voter_polarization_highlights_df["Democratic agreement"] = (
    #     voter_polarization_highlights_df.apply (lambda row: f"{row['Democratic agreement']:.1%}", axis=1))
    # voter_polarization_highlights_df["Republican agreement"] = (
    #     voter_polarization_highlights_df.apply (lambda row: f"{row['Republican agreement']:.1%}", axis=1))
    voter_polarization_highlights_df = voter_polarization_highlights_df.drop(
        columns = ["abs polarization", "Party agreement", "Key"])
    display(voter_polarization_highlights_df[:2])
    export_df("voter_polarization_highlights_df")


,Name,Party,Seat,Polarization,Democratic agreement,Republican agreement
273,A. Bell of Yarmouth,Democratic,H-47,-0.836275,0.981061,0.087121
317,R. LaRochelle of Augusta,Democratic,H-86,-0.746854,1.000000,0.197183


In [44]:
if True: # table for dash sorting:
    # For sorting: name, city, district (duplicate) (remove polarization column)
    # 
    # display(voter_polarization_df[:2])
    # voter_polarization_dash_table_df = voter_polarization_df.copy()
    new_rows = []
    dash_table_lookup = defaultdict(dict)
    print(voter_polarization_df.columns)
    for index, row in voter_polarization_df.iterrows():
        if row["Seat"].find("/") > -1:
            seat_list = row["Seat"].split("/")
            for seat in seat_list:
                new_row = row.copy()
                new_row["Seat"] = seat
                new_rows.append(new_row)
        else:
            new_row = row.copy()
            new_rows.append(new_row)
    new_rows_df = pd.concat(new_rows, axis=1)
    voter_polarization_dash_table_df = new_rows_df.transpose()
    voter_polarization_dash_table_df["City"] = (
        voter_polarization_dash_table_df.apply (lambda row: legislators.lookup_city(row["Key"]), axis=1))
    voter_polarization_dash_table_df["Name"] = (
        voter_polarization_dash_table_df.apply (
            lambda row: legislators.formatted_name(row["Key"], type="name_and_initial"), axis=1))
    voter_polarization_dash_table_df["Party"] = (
        voter_polarization_dash_table_df.apply (
            lambda row: row["Party"].replace("/", "/ "), axis=1))
    for _, row in voter_polarization_dash_table_df.iterrows():
        dash_table_lookup[f"{row['Name']} of {row['City']}"]["Key"] = row["Key"]
        dash_table_lookup[f"{row['Name']} of {row['City']}"]["full_name_and_city"] = (
            legislators.formatted_name(row["Key"], "full_name_and_city"))
    # print("new_rows_df")
    # display(new_rows_df[:2])
    # voter_polarization_dash_table_df = pd.concat([voter_polarization_dash_table_df, new_rows_df])
    voter_polarization_dash_table_columns = [
        "Name", "City", "Seat", "Party", "Polarization", "Democratic agreement", "Republican agreement"]
    # voter_polarization_dash_table_df["Polarization"] = (
    #     voter_polarization_dash_table_df.apply (lambda row: f"{row['Polarization']:.1%}", axis=1))
    # voter_polarization_dash_table_df["Democratic agreement"] = (
    #     voter_polarization_dash_table_df.apply (lambda row: f"{row['Democratic agreement']:.1%}", axis=1))
    # voter_polarization_dash_table_df["Republican agreement"] = (
    #     voter_polarization_dash_table_df.apply (lambda row: f"{row['Republican agreement']:.1%}", axis=1))

    # voter_polarization_dash_table_df["Polarization"] = (
    #     voter_polarization_dash_table_df.apply (lambda row: round(row['Polarization'],3), axis=1))
    # voter_polarization_dash_table_df["Democratic agreement"] = (
    #     voter_polarization_dash_table_df.apply (lambda row: round(row['Democratic agreement'],3), axis=1))
    # voter_polarization_dash_table_df["Republican agreement"] = (
    #     voter_polarization_dash_table_df.apply (lambda row: round(row['Republican agreement'],3), axis=1))
    voter_polarization_dash_table_df = voter_polarization_dash_table_df[voter_polarization_dash_table_columns]
    display(voter_polarization_dash_table_df[:2])
    display(voter_polarization_dash_table_df[-2:])
    voter_polarization_dash_table_df = voter_polarization_dash_table_df.drop_duplicates()
    export_df("voter_polarization_dash_table_df")
    export_dict("dash_table_lookup")

    # print(dash_table_lookup)

Index(['Key', 'Name', 'Party', 'Seat', 'Polarization', 'Party agreement',
       'Democratic agreement', 'Republican agreement', 'abs polarization'],
      dtype='object')


,Name,City,Seat,Party,Polarization,Democratic agreement,Republican agreement
110,"Higgins, N.",Dover-Foxcroft,H-120,Independent,0.00793,0.597107,0.619048
171,"Stanley, P.",Medway,H-143,Republican,-0.108973,0.639842,0.528402


,Name,City,Seat,Party,Polarization,Democratic agreement,Republican agreement
286,"Gifford, J.",Lincoln,H-142,Republican,0.833945,0.051948,0.952381
273,"Bell, A.",Yarmouth,H-47,Democratic,-0.836275,0.981061,0.087121


In [45]:
if True: # selected voting statistics
    # Polarization
    # Democratic Unanimity; Republican Unanimity
    # 100% left yes
    # 100% right yes
    # Polarization < 50%, < 10%

    # ic| vote_polarization_df_copy.columns: Index
    # (['Party', 'Polarization', 'Unanimity', 'Yes polarization',
    # 'Republican yes votes', 'Republican no votes', 'Democratic yes votes',
    # 'Democratic no votes', 'Republican yes', 'Democratic yes',
    # 'Party polarization', 'Title', 'title_w_breaks', 'Date', 'Session',
    # 'Key', 'Democratic votes yes/no', 'Republican votes yes/no'],
    display(vote_polarization_df_copy.describe())
    display(vote_polarization_df_copy[:5])
    ic(vote_polarization_df_copy["Party"].unique())
    ic(vote_polarization_df_copy.columns)
    ic(len(vote_polarization_df_copy))
    sessions = sorted(vote_polarization_df_copy["Session"].unique().tolist())
    session_votes = [
        len(vote_polarization_df_copy[
            (vote_polarization_df_copy["Session"] == session)
            & (vote_polarization_df_copy["Party"] == "Republican")])
        for session in sessions]
    columns = ["Statistic"]
    columns.extend([f"Session {session}" for session in sessions])
    ic(columns)
    data = []

    row = ["Percentage bipartisan votes (polarization < 10%)"]
    row.extend([
        len(vote_polarization_df_copy[
            (vote_polarization_df_copy["Party"] == "Republican")
            & (vote_polarization_df_copy["Session"] == session)
            & (vote_polarization_df_copy["Polarization"] < 0.1)])/
            votes_per_session
        for session, votes_per_session in zip(sessions, session_votes)])
    data.append(row)

    row = ["Percentage polarized votes (polarization > 90%)"]
    row.extend([
        len(vote_polarization_df_copy[
            (vote_polarization_df_copy["Party"] == "Republican")
            & (vote_polarization_df_copy["Session"] == session)
            & (vote_polarization_df_copy["Polarization"] > 0.9)])/
            votes_per_session
        for session, votes_per_session in zip(sessions, session_votes)])
    data.append(row)
    
    row = ["Average polarization"]
    row.extend([
        vote_polarization_df_copy[
            (vote_polarization_df_copy["Party"] == "Republican")
            & (vote_polarization_df_copy["Session"] == session)]["Polarization"].mean()
        for session in sessions])
    data.append(row)

    row = ["Average Democratic unanimity"]
    row.extend([
        vote_polarization_df_copy[
            (vote_polarization_df_copy["Party"] == "Democratic")
            & (vote_polarization_df_copy["Session"] == session)]["Unanimity"].mean()
        for session in sessions])
    data.append(row)

    row = ["Average Republican unanimity"]
    row.extend([
        vote_polarization_df_copy[
            (vote_polarization_df_copy["Party"] == "Republican")
            & (vote_polarization_df_copy["Session"] == session)]["Unanimity"].mean()
        for session in sessions])
    data.append(row)

    # row = ["Percentage almost unanimous Democratic votes (unanimity > 90%)"]
    # row.extend([
    #     len(vote_polarization_df_copy[
    #         (vote_polarization_df_copy["Party"] == "Democratic")
    #         & (vote_polarization_df_copy["Session"] == session)
    #         & (vote_polarization_df_copy["Unanimity"] > 0.9)])/
    #         votes_per_session
    #     for session, votes_per_session in zip(sessions, session_votes)])
    # data.append(row)

    # row = ["Percentage almost unanimous Republican votes (unanimity > 90%)"]
    # row.extend([
    #     len(vote_polarization_df_copy[
    #         (vote_polarization_df_copy["Party"] == "Republican")
    #         & (vote_polarization_df_copy["Session"] == session)
    #         & (vote_polarization_df_copy["Unanimity"] > 0.9)])/
    #         votes_per_session
    #     for session, votes_per_session in zip(sessions, session_votes)])
    # data.append(row)

    # row = ["Democratic polarization"]
    # # df = df["Party"]
    # row.extend([
    #     vote_polarization_df_copy[
    #         vote_polarization_df_copy["Party"] == "Democratic"
    #         & vote_polarization_df_copy["Session"] == session]["Party polarization"].mean()
    #     for session in sessions])
    # data.append(row)
    # import plotly.express as px
    import plotly
    ic(sessions)
    ic(session_votes)
    df = pd.DataFrame(columns=columns, data=data)
    display(df)
    voting_statistics_df = df
    export_df("voting_statistics_df")

,Polarization,Unanimity,Yes polarization,Republican yes votes,Republican no votes,Democratic yes votes,Democratic no votes,Republican yes,Democratic yes,Party polarization
count,3.078000e+03,3078.000000,3078.000000,3078.000000,3078.000000,3078.000000,3078.000000,3078.000000,3078.000000,3078.000000
mean,2.885570e-19,0.935580,-0.473724,12.266407,25.649773,41.554256,5.710201,0.372566,0.846291,-0.473724
std,7.763241e-01,0.110585,0.614973,17.214258,24.292180,28.842385,13.661033,0.415737,0.308226,0.614973
min,-1.000000e+00,0.500000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000
25%,-8.767442e-01,0.923077,-0.954371,0.000000,3.000000,17.000000,0.000000,0.000000,0.909091,-0.954371
50%,0.000000e+00,1.000000,-0.827839,5.000000,13.000000,22.000000,0.000000,0.114286,1.000000,-0.827839
75%,8.767442e-01,1.000000,0.000000,16.000000,52.000000,70.000000,3.000000,0.888889,1.000000,0.000000
max,1.000000e+00,1.000000,1.000000,70.000000,71.000000,85.000000,81.000000,1.000000,1.000000,1.000000


,Party,Polarization,Unanimity,Yes polarization,Republican yes votes,Republican no votes,Democratic yes votes,Democratic no votes,Republican yes,Democratic yes,Party polarization,Title,title_w_breaks,Date,Session,Key,Democratic votes yes/no,Republican votes yes/no
3077,Republican,1.000000,1.000000,-1.000000,0,12,18,0,0.000000,1.000000,-1.000000,"Resolve, To Establish A Pilot Program To Encou...","Resolve, To Establish A Pilot<br>Program To En...",2022-03-09,130,2772e24a-9d5d-4edb-a2d3-5e99ed340509,18/0,0/12
2599,Republican,0.893333,1.000000,-0.893333,0,62,67,8,0.000000,0.893333,-0.893333,An Act To Amend The Maine Bail Code Regarding ...,An Act To Amend The Maine Bail<br>Code Regardi...,2021-06-30,130,1a355a89-f3fd-4a32-a055-bee63c8eedc9,67/8,0/62
2975,Republican,0.888889,1.000000,-0.888889,0,11,16,2,0.000000,0.888889,-0.888889,An Act Implementing The Recommendations Of The...,An Act Implementing The<br>Recommendations Of ...,2022-04-25,130,001d68e6-c434-4b52-a2d4-b3856136f36e,16/2,0/11
2704,Democratic,-0.890563,0.975309,-0.890563,5,54,79,2,0.084746,0.975309,-0.890563,An Act To Provide For School Choice And Online...,An Act To Provide For School<br>Choice And Onl...,2021-04-28,130,d36e809e-a1f3-474e-9eeb-cb10b82ebdb7,79/2,5/54
2705,Republican,0.890563,0.915254,-0.890563,5,54,79,2,0.084746,0.975309,-0.890563,An Act To Provide For School Choice And Online...,An Act To Provide For School<br>Choice And Onl...,2021-04-28,130,d36e809e-a1f3-474e-9eeb-cb10b82ebdb7,79/2,5/54


ic| vote_polarization_df_copy["Party"].unique(): array(['Republican', 'Democratic'], dtype=object)
ic| vote_polarization_df_copy.columns: Index(['Party', 'Polarization', 'Unanimity', 'Yes polarization',
                                              'Republican yes votes', 'Republican no votes', 'Democratic yes votes',
                                              'Democratic no votes', 'Republican yes', 'Democratic yes',
                                              'Party polarization', 'Title', 'title_w_breaks', 'Date', 'Session',
                                              'Key', 'Democratic votes yes/no', 'Republican votes yes/no'],
                                             dtype='object')
ic| len(vote_polarization_df_copy): 3078
ic| columns: ['Statistic', 'Session 128', 'Session 129', 'Session 130']
ic| sessions: ['128', '129', '130']
ic| session_votes: [627, 352, 560]


,Statistic,Session 128,Session 129,Session 130
0,Percentage bipartisan votes (polarization < 10%),0.192982,0.136364,0.098214
1,Percentage polarized votes (polarization > 90%),0.322169,0.465909,0.607143
2,Average polarization,0.596359,0.709988,0.776835
3,Average Democratic unanimity,0.958427,0.949060,0.949111
4,Average Republican unanimity,0.888075,0.921215,0.950213


In [46]:
if False: # histograms of polarization
    import plotly.graph_objects as go

    histogram_data = []
    print("polarization")
    for session in sessions:
        # fig = px.histogram(vote_polarization_df_copy[
        #     (vote_polarization_df_copy["Party"] == "Republican")
        #     & (vote_polarization_df_copy["Session"] == session)], 
        #     x = "Polarization",
        #     histnorm="percent",
        #     )
        # fig.show()
        count, index = np.histogram(
                vote_polarization_df_copy[
                    (vote_polarization_df_copy["Party"] == "Republican")
                    & (vote_polarization_df_copy["Session"] == session)]["Polarization"],
                bins=20,
                density=True,
            )
        histogram_data.append([count, index])

        # plotly, go.Scatter with line shape set to 'hvh'
        fig = go.Figure()
        fig.add_traces(go.Scatter(x=index, y = count,
                                line=dict(width = 1, shape='hvh')))
        fig.show()

    histogram_data = []
    print("Republican Unanimity")
    for session in sessions:
        count, index = np.histogram(
                vote_polarization_df_copy[
                    (vote_polarization_df_copy["Party"] == "Republican")
                    & (vote_polarization_df_copy["Session"] == session)]["Unanimity"],
                bins=20,
                density=True,
            )
        histogram_data.append([count, index])

        # plotly, go.Scatter with line shape set to 'hvh'
        fig = go.Figure()
        fig.add_traces(go.Scatter(x=index, y = count,
                                line=dict(width = 1, shape='hvh')))
        fig.show()

    histogram_data = []
    print("Democratic Unanimity")
    for session in sessions:
        count, index = np.histogram(
                vote_polarization_df_copy[
                    (vote_polarization_df_copy["Party"] == "Democratic")
                    & (vote_polarization_df_copy["Session"] == session)]["Unanimity"],
                bins=20,
                density=True,
            )
        histogram_data.append([count, index])

        # plotly, go.Scatter with line shape set to 'hvh'
        fig = go.Figure()
        fig.add_traces(go.Scatter(x=index, y = count,
                                line=dict(width = 1, shape='hvh')))
        fig.show()

        # recommended cosmetics

        # y-axis cosmetics
        # fig.update_yaxes(
        #     showgrid=False,
        #     ticks="inside",
        #     tickson="boundaries",
        #     ticklen=10,
        #     showline=True,
        #     linewidth=1,
        #     linecolor='black',
        #     mirror=True,
        #     zeroline=False)

        # x-axis cosmetics
        # fig.update_xaxes(
        #     showgrid=False,
        #     ticks="inside",
        #     tickson="boundaries",
        #     ticklen=10,
        #     showline=True,
        #     linewidth=1,
        #     linecolor='black',
        #     mirror=True,
        #     zeroline=False)
        pass

In [47]:
if True: # process Bangor Daily News canidate lists OF
    import csv
    import pprint
    from collections import defaultdict

    # @TODO: merge the following 
    senate_dict = defaultdict(list)
    with open('maine_2022_candidates.csv') as _file:
        _reader = csv.reader(_file)
        for row in _reader:
            # print(row)
            cell = row[0]
            if cell.find("District") > -1:
                senate_district = int(cell.split("District ")[-1])
            elif len(cell) > 0 and cell.find("CANDIDATES") == -1:
                senate_dict[senate_district].append(
                    {"name": row[1], "party":row[0]})
    # pprint.pprint(senate_dict)

    house_dict = defaultdict(list)
    with open('maine_2022_candidates.csv') as _file:
        _reader = csv.reader(_file)
        for row in _reader:
            # print(row)
            cell = row[4]
            if cell.find("District") > -1:
                house_district = int(cell.split("District ")[-1])
            elif len(cell) > 0 and cell.find("CANDIDATES") == -1:
                house_dict[house_district].append(
                    {"name": row[5], "party":row[4]})
    # pprint.pprint(house_dict)
    print(f"Read {len(senate_dict)} Senate races.")
    print(f"Read {len(house_dict)} House races.")

    jr_list = []
    uuid = "140459ab-ba63-410a-819c-aec28c2f2f66"
    BDN_uuid_key_to_name = {}
    def process_legislative_dict(legislative_dict):
        for district, candidates in legislative_dict.items():
            for candidate in candidates:
                names = candidate["name"].upper().split(" ")
                first_name = names[0]
                last_name = names[-1]
                if last_name[:2] == "JR":
                    jr_list.append(names)
                if last_name == "III" or last_name[:2] == "JR":
                    last_name = names[-2]
                if first_name == "JOE":
                    first_name = "JOSEPH"
                if first_name == "JIM":
                    first_name = "JAMES"
                if first_name[-1] == ".":
                    first_name = first_name[:-1]
                lookup = legislators._lookup[last_name]
                if lookup:
                    for legislator_key in lookup:
                        if (legislator_key.upper().find(first_name) > -1):
                            # and legislator_key.upper().find(" OF ") > -1):
                            if "key_list" in candidate:
                                candidate["key_list"].add(
                                    f"{legislator_key}-{candidate['party']}")
                            else:
                                candidate["key_list"] = set([
                                    f"{legislator_key}-{candidate['party']}"])
                            BDN_uuid_key_to_name[legislator_key] = candidate["name"]
                        
                    if "key_list" not in candidate:
                        print(
                            f"imperfect match: {first_name} {last_name} : "
                            f"{[key[:-(len(uuid)+1)] for key in lookup]}")
                    # print(first_name, last_name, legislators._lookup[last_name.upper()])
                    pass
    process_legislative_dict(senate_dict)
    process_legislative_dict(house_dict)
    # print(f"\nJr list: {jr_list}")

    candidate_keys_list = []
    for district, candidates in senate_dict.items():
        candidate_keys = []
        for candidate in candidates:
            if "key_list" in candidate:
                candidate_keys.append(candidate)
        if len(candidate_keys) > 1:
            candidate_keys_list.append([f"S-{district}", candidate_keys])
    for district, candidates in house_dict.items():
        candidate_keys = []
        for candidate in candidates:
            if "key_list" in candidate:
                candidate_keys.append(candidate)
        if len(candidate_keys) > 1:
            candidate_keys_list.append([f"H-{district}", candidate_keys])


    print(f"\nRaces of interest: {[item[0] for item in candidate_keys_list]}\n")
    
    columns = ["race", "name", "party", "key"]
    data = []
    exclusion_set = {"H-22", "H-57", "H-68", "H-96", "S-5", "S-31"}

    for item in candidate_keys_list:
        if item[0] not in exclusion_set:
            print(f"{item[0]}: ", end="")
            for matches in item[1]:
                row=[item[0]]
                for match in matches.values():
                    if type(match) == set:
                        key = next(iter(match))[:-2]
                        result = legislators.inverse_lookup(key)
                        row.append(result)
                        print(f"{result}, ", end="")
                    else:
                        print(f"{match}, ", end="")
                        row.append(match)
                data.append(row)
            print()
    races_of_interest_2022 = pd.DataFrame(data=data, columns=columns)
    display(races_of_interest_2022[:3])
    # pprint.pprint(data)
                # print(f"{legislators.inverse_lookup(match[:-2])}, ", end="")



Read 35 Senate races.
Read 151 House races.
imperfect match: MICHAEL TIPPING : ['Ryan-Tipping']
imperfect match: JAMES LIBBY : ['Laurel-D-Libby', 'Aaron-F-Libby', 'Nathan-L-Libby']
imperfect match: JENNIFER WHITE : ['Bruce-A-White', 'Dustin-Michael-White']
imperfect match: KENDRA WILLIAMS : ['Lynne-A-Williams']
imperfect match: BRADLEY DUCHARME : ['John-E-Ducharme-III']
imperfect match: AUSTIN THERIAULT : ['Charles-Kenneth-Theriault', 'Timothy-S-Theriault']
imperfect match: KENNETH DAVIS : ['Paul-T-Davis-Sr', 'Paul-T-Davis']
imperfect match: RON RUSSELL : ['Diane-M-Russell', 'Diane-Russell']
imperfect match: PETER CROCKETT : ['W-Edward-Crockett']
imperfect match: IRENE GIFFORD : ['Jeffery-Allen-Gifford']
imperfect match: JAMES WHITE : ['Bruce-A-White', 'Dustin-Michael-White']
imperfect match: CHAD PERKINS : ['Michael-D-Perkins']
imperfect match: TRICIA WARREN : ['Charlotte-Warren', 'Sophia-B-Warren']
imperfect match: ALLISON PERKINS : ['Michael-D-Perkins']
imperfect match: CALDWELL JAC

,race,name,party,key
0,S-1,Susan Bernard,R,BERNARD OF CARIBOU
1,S-1,Troy D. Jackson,D,JACKSON OF AROOSTOOK
2,S-7,Brian Langley,R,LANGLEY OF HANCOCK


In [48]:
if True: # 2022 highlights
   data = []
   columns = ['Seat', 'Name', 'Party', 'Polarization', 
              'Democratic agreement', 'Republican agreement',]
   for _, row in races_of_interest_2022.iterrows():
      new_row = [row["race"], row["name"], row["party"]]
      if type(row['key']) == set:
         row['key'] = next(iter(row['key']))
      match_row = voter_polarization_df[voter_polarization_df["Key"]==row["key"]]
      if len(match_row) > 0:
         new_row.append(match_row.iloc[0]["Polarization"])
         new_row.append(match_row.iloc[0]["Democratic agreement"])
         new_row.append(match_row.iloc[0]["Republican agreement"])
         data.append(new_row)
      else:
         print("match_row/row['key']", match_row, row["key"])
         print(row)
         print(f"type(key): {type(row['key'])}")
         # for key in row["key"]:
         #    print(f"key loop: {key}")
         #    print(f"key loop: {legislators.lookup(key)['name']}")
         # print(legislators.lookup(next(iter(row["key"]))))
      # print(new_row)
   head_to_head_df = pd.DataFrame(data=data, columns=columns)
   # head_to_head_df = head_to_head_df.sort_values(
   #    by=['Seat',  'Polarization'])

   display(head_to_head_df)
   export_df("head_to_head_df")



match_row/row['key'] Empty DataFrame
Columns: [Key, Name, Party, Seat, Polarization, Party agreement, Democratic agreement, Republican agreement, abs polarization]
Index: [] HICKMAN
race                 S-14
name     Craig V. Hickman
party                   D
key               HICKMAN
Name: 6, dtype: object
type(key): <class 'str'>


,Seat,Name,Party,Polarization,Democratic agreement,Republican agreement
0,S-1,Susan Bernard,R,0.802667,0.101266,0.974684
1,S-1,Troy D. Jackson,D,-0.575697,0.968320,0.340751
2,S-7,Brian Langley,R,0.344068,0.501931,0.922481
3,S-7,Nicole C. Grohoski,D,-0.723254,0.936916,0.151869
4,S-11,Glenn Chip Curry,D,-0.634546,0.957746,0.270463
5,S-11,MaryAnne Kinney,R,0.652286,0.214586,0.941011
6,S-14,Jeffery P. Hanley,R,0.703637,0.159280,0.943135
7,S-20,Bettyann W. Sheats,D,-0.634242,0.956190,0.267176
8,S-20,Eric L. Brakey,R,0.396371,0.372624,0.793103
9,H-56,Kent Ackley,U,-0.595445,0.895038,0.258126


In [49]:
if True: # export all_voters_table
    BDN_short_key_to_name = {}
    for key, value in BDN_uuid_key_to_name.items():
        lookup = legislators.inverse_lookup(key)
        if type(lookup) == set:
            lookup = next(iter(lookup))
        BDN_short_key_to_name[lookup] = value
    voter_polarization_df["BDN_Name"] = voter_polarization_df.apply(
                lambda row: BDN_short_key_to_name.get(row["Key"], "NA"), axis=1)
    all_voters_table = voter_polarization_df[[
        "Key", "Name", "BDN_Name", "Seat", "Party", "Polarization",
        "Democratic agreement", "Republican agreement"]]
    all_voters_table = all_voters_table.rename(columns={"Key": "Voter key"})
    export_df(None, all_voters_table, "all_voters_table", index=False)
    display(all_voters_table)



,Voter key,Name,BDN_Name,Seat,Party,Polarization,Democratic agreement,Republican agreement
110,HIGGINS OF DOVERFOXCROFT,N. Higgins of Dover-Foxcroft,NA,H-120,Independent,0.007930,0.597107,0.619048
171,STANLEY OF MEDWAY,P. Stanley of Medway,NA,H-143,Republican,-0.108973,0.639842,0.528402
146,NADEAU OF WINSLOW,C. Nadeau of Winslow,NA,H-78,Republican,-0.129852,0.661479,0.516883
219,SAVIELLO OF FRANKLIN,T. Saviello of Franklin,NA,S-17,Republican,0.205928,0.608209,0.838346
215,KATZ OF KENNEBEC,R. Katz of Kennebec,NA,S-15,Republican,0.216361,0.578182,0.816850
...,...,...,...,...,...,...,...,...
298,RIELLY OF WESTBROOK,M. Rielly of Westbrook,Morgan Rielly,H-34,Democratic,-0.828353,0.980989,0.095057
295,OSHER OF ORONO,L. Osher of Orono,Laurie Osher,H-123,Democratic,-0.828969,0.980469,0.093750
291,LOOKNER OF PORTLAND,G. Lookner of Portland,Grayson B. Lookner,H-37,Democratic,-0.831795,0.976378,0.074803
286,GIFFORD OF LINCOLN,J. Gifford of Lincoln,NA,H-12/H-142,Republican,0.833945,0.051948,0.952381


In [50]:
finished_successfully = {}
export_dict("finished_successfully")